# Data ETL

## Imports and Setup

In [ ]:
import pandas as pd
import openpyxl
from datetime import datetime
import os
import numpy as np
from statsmodels.tsa.seasonal import STL
from scipy import interpolate
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr


In [ ]:

# Path to your Excel file
file_path = 'data/raw/Bloomberg_Data.xlsx'

# Define which sheets use column C instead of column B
use_column_c = [
    "US_Building_Permits",
    "US _BP_Single_Housing",
    "US_Housing_Start",
    "US_New_Home_Sales",
    "US_Existing_Home _Sales",
    "US Existing_Single_Home_Sales",
    "CAD_Housing_Start"
]

# Define sheets to ignore
sheets_to_ignore = [
    "US_Population_Growth_Rate_Bloom"
]

## Function Definitions

### Core Data Processing Functions


In [ ]:
def normalize_date(date):
    """Normalize date to end of month, with special handling for quarterly data"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    # Get the last day of the month
    year = date.year
    month = date.month
    
    # Create end of month date
    if month == 12:
        end_of_month = datetime(year, 12, 31)
    else:
        end_of_month = datetime(year, month + 1, 1) - pd.Timedelta(days=1)
    
    return end_of_month.date()

def normalize_quarterly_date(date):
    """Normalize quarterly date to end of quarter"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    year = date.year
    month = date.month
    
    # Map to end of quarter
    if month in [1, 2, 3]:  # Q1
        return datetime(year, 3, 31).date()
    elif month in [4, 5, 6]:  # Q2
        return datetime(year, 6, 30).date()
    elif month in [7, 8, 9]:  # Q3
        return datetime(year, 9, 30).date()
    else:  # Q4
        return datetime(year, 12, 31).date()

def extract_sheet_data(file_path, sheet_name, use_col_c):
    """Extract data from a specific sheet with improved data cleaning"""
    # Determine which column to use
    data_column = 'C' if sheet_name in use_col_c else 'B'
    
    # Read the sheet starting from row 7 (index 6 in pandas)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Extract dates from column A and values from the appropriate column
    # Row 7 in Excel is index 6 in pandas (0-indexed)
    dates = df.iloc[6:, 0]  # Column A, starting from row 7
    
    if data_column == 'C':
        values = df.iloc[6:, 2]  # Column C
    else:
        values = df.iloc[6:, 1]  # Column B
    
    # Create a temporary dataframe
    temp_df = pd.DataFrame({
        'date': dates,
        'value': values
    })
    
    # Remove rows where value is NaN or empty
    temp_df = temp_df.dropna(subset=['value'])
    
    # Remove rows where date is NaN
    temp_df = temp_df.dropna(subset=['date'])
    
    # Determine the appropriate date normalization based on sheet name
    # Population growth data needs special quarterly normalization
    if sheet_name == 'US_Population_Growth_Rate_FRED':
        temp_df['date'] = temp_df['date'].apply(normalize_quarterly_date)
    else:
        temp_df['date'] = temp_df['date'].apply(normalize_date)
    
    # Remove any rows where date normalization failed
    temp_df = temp_df.dropna(subset=['date'])
    
    # Rename value column to sheet name
    temp_df = temp_df.rename(columns={'value': sheet_name})
    
    return temp_df

def is_row_worth_keeping(row, important_columns, min_important_values=5):
    """
    Determine if a row is worth keeping based on the number of important values.
    A row is worth keeping if it has at least min_important_values non-null values
    in important columns (excluding CPI-only rows).
    """
    # Count non-null values in important columns
    non_null_count = row[important_columns].notna().sum()
    
    # Special case: if the row only has CPI data, drop it
    if non_null_count == 0 and row.get('US_CPI') is not None:
        return False
    
    # Keep rows with sufficient important data
    return non_null_count >= min_important_values

### Canada-US Softwood Lumber Exports Data Processing Functions

In [ ]:
def extract_softwood_data(file_path):
    """Extract and process CAD_Softwood_Export_to_US data with STL decomposition for missing values"""
    
    # Read the softwood export sheet
    df = pd.read_excel(file_path, sheet_name='CAD_Softwood_Export_to_US', header=None)
    
    # Extract dates and values (data starts at row 6, index 6)
    dates = df.iloc[6:, 0]  # Column A
    values = df.iloc[6:, 1]  # Column B
    
    # Create dataframe
    softwood_df = pd.DataFrame({
        'date': dates,
        'value': values
    })
    
    # Remove rows where both date and value are NaN
    softwood_df = softwood_df.dropna(subset=['date'])
    
    # Convert dates to datetime and normalize to end of month
    softwood_df['date'] = pd.to_datetime(softwood_df['date'])
    softwood_df['date'] = softwood_df['date'].apply(normalize_date)
    
    # Remove any rows where date normalization failed
    softwood_df = softwood_df.dropna(subset=['date'])
    
    # Sort by date
    softwood_df = softwood_df.sort_values('date').reset_index(drop=True)
    
    print(f"Extracted {len(softwood_df)} monthly data points")
    print(f"Date range: {softwood_df['date'].min()} to {softwood_df['date'].max()}")
    print(f"Missing values: {softwood_df['value'].isna().sum()}")
    
    return softwood_df

def impute_missing_values_stl(df):
    """Impute missing values using STL decomposition with seasonal interpolation"""
    
    # Convert values to numeric to ensure proper data type
    df_copy = df.copy()
    df_copy['value'] = pd.to_numeric(df_copy['value'], errors='coerce')
    
    # Create a complete date range for monthly data
    start_date = df_copy['date'].min()
    end_date = df_copy['date'].max()
    complete_dates = pd.date_range(start=start_date, end=end_date, freq='ME')  # Use 'ME' instead of 'M'
    complete_dates = [normalize_date(d) for d in complete_dates]
    
    # Create complete dataframe
    complete_df = pd.DataFrame({'date': complete_dates})
    complete_df = complete_df.merge(df_copy, on='date', how='left')
    
    # Check if we have enough data for STL decomposition
    non_null_count = complete_df['value'].notna().sum()
    total_count = len(complete_df)
    
    print(f"Data completeness: {non_null_count}/{total_count} ({non_null_count/total_count*100:.1f}%)")
    
    if non_null_count < 24:  # Need at least 2 years of data for STL
        print("Warning: Insufficient data for STL decomposition. Using linear interpolation instead.")
        complete_df['value'] = complete_df['value'].interpolate(method='linear')
    else:
        # Prepare data for STL decomposition
        complete_df = complete_df.set_index('date')
        
        # Store original missing mask before filling
        original_missing_mask = complete_df['value'].isna()
        
        # Forward fill and backward fill to handle edge cases for STL
        ts_filled = complete_df['value'].ffill().bfill()
        
        # Perform STL decomposition
        try:
            # Use the working parameters: seasonal=11, period=12
            stl = STL(ts_filled, seasonal=11, period=12, robust=True)
            result = stl.fit()
            
            # Use seasonal component for interpolation of missing values
            seasonal_component = result.seasonal
            trend_component = result.trend
            residual_component = result.resid
            
            # For missing values, use trend + seasonal components
            if original_missing_mask.any():
                # Fill missing values with trend + seasonal
                complete_df.loc[original_missing_mask, 'value'] = (
                    trend_component[original_missing_mask] + 
                    seasonal_component[original_missing_mask]
                )
            
            print("STL decomposition completed successfully")
            print(f"Imputed {original_missing_mask.sum()} missing values using STL")
            
            # Show some statistics
            print(f"STL Statistics - Trend range: {trend_component.min():.0f} to {trend_component.max():.0f}")
            print(f"STL Statistics - Seasonal range: {seasonal_component.min():.0f} to {seasonal_component.max():.0f}")
            
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Falling back to linear interpolation")
            complete_df['value'] = complete_df['value'].interpolate(method='linear')
    
    # Reset index and return
    complete_df = complete_df.reset_index()
    return complete_df

def aggregate_monthly_to_quarterly(df):
    """Aggregate monthly data to quarterly data"""
    
    # Convert date column to datetime for proper resampling
    df_copy = df.copy()
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    
    # Set date as index for resampling
    df_indexed = df_copy.set_index('date')
    
    # Resample to quarterly (end of quarter) and sum the values
    quarterly_df = df_indexed.resample('QE').sum().reset_index()  # Use 'QE' instead of 'Q'
    
    # Convert quarterly dates to end of quarter format
    quarterly_df['date'] = quarterly_df['date'].apply(normalize_quarterly_date)
    
    # Rename the value column
    quarterly_df = quarterly_df.rename(columns={'value': 'CAD_Softwood_Export_to_US'})
    
    print(f"Aggregated to {len(quarterly_df)} quarterly data points")
    print(f"Quarterly date range: {quarterly_df['date'].min()} to {quarterly_df['date'].max()}")
    
    return quarterly_df

def impute_master_df_softwood_stl(master_df):
    """
    Impute missing values in CAD_Softwood_Export_to_US using STL decomposition.
    Handles edge missing values by extrapolating trend + seasonal components.
    
    Parameters:
    -----------
    master_df : pandas.DataFrame
        Master dataframe with 'Date' and 'CAD_Softwood_Export_to_US' columns
        
    Returns:
    --------
    pandas.DataFrame
        Updated dataframe with imputed softwood values
    """
    
    print("\n" + "="*60)
    print("STL Imputation for Master DataFrame Softwood Values")
    print("="*60)
    
    # Create a copy to avoid modifying the original
    df_copy = master_df.copy()
    
    # Extract Date and CAD_Softwood_Export_to_US columns
    softwood_data = df_copy[['Date', 'CAD_Softwood_Export_to_US']].copy()
    
    # Convert Date to datetime and sort
    softwood_data['Date'] = pd.to_datetime(softwood_data['Date'])
    softwood_data = softwood_data.sort_values('Date').reset_index(drop=True)
    
    # Check initial missing values
    initial_missing = softwood_data['CAD_Softwood_Export_to_US'].isna().sum()
    total_values = len(softwood_data)
    
    print(f"Initial analysis:")
    print(f"- Total data points: {total_values}")
    print(f"- Missing values: {initial_missing} ({initial_missing/total_values*100:.1f}%)")
    print(f"- Date range: {softwood_data['Date'].min().date()} to {softwood_data['Date'].max().date()}")
    
    if initial_missing == 0:
        print("No missing values found. Returning original dataframe.")
        return df_copy
    
    # Convert values to numeric
    softwood_data['CAD_Softwood_Export_to_US'] = pd.to_numeric(
        softwood_data['CAD_Softwood_Export_to_US'], errors='coerce'
    )
    
    # Check if we have enough data for STL decomposition
    non_null_count = softwood_data['CAD_Softwood_Export_to_US'].notna().sum()
    
    print(f"\nSTL Decomposition Setup:")
    print(f"- Non-null values: {non_null_count}")
    print(f"- Data completeness: {non_null_count/total_values*100:.1f}%")
    
    if non_null_count < 8:  # Need at least 2 years of quarterly data for STL
        print("Warning: Insufficient data for STL decomposition. Using linear interpolation instead.")
        softwood_data['CAD_Softwood_Export_to_US'] = softwood_data['CAD_Softwood_Export_to_US'].interpolate(method='linear')
    else:
        # Prepare data for STL decomposition
        softwood_ts = softwood_data.set_index('Date')['CAD_Softwood_Export_to_US']
        
        # Store original missing mask
        original_missing_mask = softwood_ts.isna()
        
        # Forward fill and backward fill to handle edge cases for STL
        ts_filled = softwood_ts.ffill().bfill()
        
        try:
            # Perform STL decomposition with quarterly parameters
            print("Performing STL decomposition...")
            print("- Parameters: seasonal=11, period=4 (quarterly), robust=True")
            
            stl = STL(ts_filled, seasonal=11, period=4, robust=True)
            result = stl.fit()
            
            # Extract components
            trend_component = result.trend
            seasonal_component = result.seasonal
            residual_component = result.resid
            
            print("STL decomposition completed successfully!")
            print(f"- Trend range: {trend_component.min():.0f} to {trend_component.max():.0f}")
            print(f"- Seasonal range: {seasonal_component.min():.0f} to {seasonal_component.max():.0f}")
            print(f"- Residual std: {residual_component.std():.0f}")
            
            # Impute missing values using trend + seasonal components
            if original_missing_mask.any():
                imputed_values = trend_component[original_missing_mask] + seasonal_component[original_missing_mask]
                softwood_ts.loc[original_missing_mask] = imputed_values
                
                print(f"\nImputation Results:")
                print(f"- Imputed {original_missing_mask.sum()} missing values")
                print(f"- Imputed value range: {imputed_values.min():.0f} to {imputed_values.max():.0f}")
                
                # Show some examples of imputed values
                imputed_indices = softwood_ts.index[original_missing_mask]
                print(f"- Sample imputed dates: {[d.date() for d in imputed_indices[:3]]}")
            
            # Update the dataframe
            softwood_data['CAD_Softwood_Export_to_US'] = softwood_ts.values
            
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Falling back to linear interpolation")
            softwood_data['CAD_Softwood_Export_to_US'] = softwood_data['CAD_Softwood_Export_to_US'].interpolate(method='linear')
    
    # Ensure Date column is in the same format as the original dataframe
    # Convert back to the original Date format (object type with date objects)
    softwood_data['Date'] = pd.to_datetime(softwood_data['Date']).dt.date
    
    # Create a mapping dictionary for imputed values to avoid merge duplicates
    imputed_mapping = dict(zip(softwood_data['Date'], softwood_data['CAD_Softwood_Export_to_US']))
    
    # Apply imputed values directly to avoid merge duplicates
    df_copy['CAD_Softwood_Export_to_US'] = df_copy['Date'].map(imputed_mapping).fillna(df_copy['CAD_Softwood_Export_to_US'])
    
    # Final verification
    final_missing = df_copy['CAD_Softwood_Export_to_US'].isna().sum()
    print(f"\nFinal Results:")
    print(f"- Missing values after imputation: {final_missing}")
    print(f"- Imputation success: {'✓' if final_missing == 0 else '✗'}")
    
    return df_copy


## Canada-US Softwood Lumber Exports Data Processing


In [ ]:
# Process CAD_Softwood_Export_to_US data
print("Processing CAD_Softwood_Export_to_US data...")
print("="*50)

# Extract monthly softwood data
softwood_monthly = extract_softwood_data(file_path)

# Impute missing values using STL decomposition
print("\nImputing missing values using STL decomposition...")
softwood_complete = impute_missing_values_stl(softwood_monthly)

# Aggregate monthly data to quarterly
print("\nAggregating monthly data to quarterly...")
softwood_quarterly = aggregate_monthly_to_quarterly(softwood_complete)

# Check for duplicate dates and remove them
print("\nChecking for duplicate dates...")
initial_count = len(softwood_quarterly)
duplicate_mask = softwood_quarterly.duplicated(subset=['date'], keep='first')
duplicate_count = duplicate_mask.sum()

if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate date(s). Removing duplicates...")
    duplicate_dates = softwood_quarterly[duplicate_mask]['date'].tolist()
    print(f"Duplicate dates: {duplicate_dates}")
    
    # Keep only the first occurrence of each date
    softwood_quarterly = softwood_quarterly[~duplicate_mask].reset_index(drop=True)
    final_count = len(softwood_quarterly)
    print(f"Removed {initial_count - final_count} duplicate row(s).")
else:
    print("No duplicate dates found.")

print(f"\nFinal softwood quarterly data: {len(softwood_quarterly)} data points")
print("Sample of processed data:")
print(softwood_quarterly.head(10))


### STL Decomposition: Mathematical Foundation and Rationale

**STL (Seasonal and Trend decomposition using Loess)** is a robust time series decomposition method that separates a time series into three components:

#### Mathematical Model
For a time series $Y_t$, STL decomposes it as:
$$Y_t = T_t + S_t + R_t$$

Where:
- **$T_t$** = Trend component (long-term movement)
- **$S_t$** = Seasonal component (recurring patterns within a year)
- **$R_t$** = Residual component (irregular fluctuations)

#### Why STL for Canada-US Softwood Lumber Exports Data?

1. **Seasonal Nature of Construction**: Softwood lumber exports exhibit strong seasonal patterns due to:
   - Construction activity peaks in spring/summer
   - Winter slowdowns in building activity
   - Weather-dependent construction cycles

2. **Robust to Outliers**: STL uses Loess (Locally Weighted Scatterplot Smoothing) which is:
   - Less sensitive to extreme values than traditional methods
   - Handles irregular patterns better than moving averages
   - Preserves local patterns while smoothing global trends

3. **Flexible Seasonal Patterns**: Unlike fixed seasonal models, STL:
   - Allows seasonal patterns to evolve over time
   - Handles changing amplitude of seasonal effects
   - Adapts to structural breaks in the data

#### Our Implementation Parameters

- **`seasonal=11`**: Uses 11-point seasonal window for monthly data
- **`period=12`**: Assumes 12-month seasonal cycle (annual pattern)
- **`robust=True`**: Uses robust statistics to handle outliers

#### Missing Value Imputation Strategy

For missing values at time $t$, we estimate:
$$\hat{Y}_t = \hat{T}_t + \hat{S}_t$$

This approach:
- Preserves the underlying seasonal structure
- Maintains trend consistency
- Provides more realistic estimates than simple interpolation
- Accounts for the specific month's typical seasonal behavior

#### Advantages Over Alternatives

- **vs. Linear Interpolation**: Captures seasonal patterns, not just linear trends
- **vs. Moving Averages**: More flexible and robust to outliers
- **vs. ARIMA**: Simpler, more interpretable, and handles missing values naturally
- **vs. Simple Seasonal Decomposition**: More robust and handles irregular patterns better


## Data Extraction

In [ ]:
# Load the Excel file to get all sheet names
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names

print(f"Found {len(sheet_names)} sheets in the Excel file\n")

# Extract data from all sheets
all_dataframes = []

# Add the processed softwood quarterly data first
print(f"Adding processed softwood data...", end=' ')
all_dataframes.append(softwood_quarterly)
print(f"✓ ({len(softwood_quarterly)} data points)")

for sheet_name in sheet_names:
    # Skip ignored sheets and the softwood sheet (already processed)
    if sheet_name in sheets_to_ignore or sheet_name == 'CAD_Softwood_Export_to_US':
        if sheet_name == 'CAD_Softwood_Export_to_US':
            print(f"Skipping: {sheet_name} (processed separately)")
        else:
            print(f"Skipping: {sheet_name} (ignored)")
        continue
    
    print(f"Processing: {sheet_name}...", end=' ')
    try:
        df = extract_sheet_data(file_path, sheet_name, use_column_c)
        all_dataframes.append(df)
        print(f"✓ ({len(df)} data points)")
    except Exception as e:
        print(f"✗ Error: {e}")


## Data Merging

In [ ]:
# Merge all dataframes on the date column
print("\nMerging all data into master dataframe...")

master_df = all_dataframes[0]
for df in all_dataframes[1:]:
    master_df = master_df.merge(df, on='date', how='outer')

# Sort by date (most recent first)
master_df = master_df.sort_values('date', ascending=False)

# Rename date column to 'Date'
master_df = master_df.rename(columns={'date': 'Date'})

# Reset index
master_df = master_df.reset_index(drop=True)

## Data Filtering

In [ ]:
print(f"\nBefore filtering:")
print(f"Total rows: {len(master_df)}")

# Identify important columns (all except Date and US_CPI)
important_cols = [col for col in master_df.columns if col not in ['Date', 'US_CPI']]

# Apply improved filtering logic
print(f"\nApplying improved filtering logic...")

# Create a mask for rows worth keeping
keep_mask = master_df.apply(lambda row: is_row_worth_keeping(row, important_cols, min_important_values=8), axis=1)

# Filter the dataframe
master_df_filtered = master_df[keep_mask].copy()

print(f"\nAfter improved filtering (minimum 8 important non-null values, excluding CPI-only rows):")
print(f"Total rows: {len(master_df_filtered)}")
print(f"Rows removed: {len(master_df) - len(master_df_filtered)}")

# Show some statistics about the filtering
print(f"\nFiltering statistics:")
print(f"- Rows with only CPI data: {len(master_df[(master_df[important_cols].notna().sum(axis=1) == 0) & (master_df['US_CPI'].notna())])}")
print(f"- Rows with 1-7 important values: {len(master_df[(master_df[important_cols].notna().sum(axis=1) >= 1) & (master_df[important_cols].notna().sum(axis=1) < 8)])}")
print(f"- Rows with 8+ important values: {len(master_df[master_df[important_cols].notna().sum(axis=1) >= 8])}")

## Data Quality Check

In [ ]:
print("\nData Quality Check - Detailed analysis of filtered data:")
check_cols = [col for col in master_df_filtered.columns if col != 'Date']
important_cols_check = [col for col in check_cols if col != 'US_CPI']

print(f"\nNon-null value distribution in filtered data:")
non_null_counts = master_df_filtered[check_cols].notna().sum(axis=1)
important_non_null_counts = master_df_filtered[important_cols_check].notna().sum(axis=1)

print(f"- Total non-null values per row: min={non_null_counts.min()}, max={non_null_counts.max()}, mean={non_null_counts.mean():.1f}")
print(f"- Important non-null values per row: min={important_non_null_counts.min()}, max={important_non_null_counts.max()}, mean={important_non_null_counts.mean():.1f}")

# Check for any remaining sparse rows
sparse_rows = []
for idx, row in master_df_filtered.iterrows():
    non_null = row[check_cols].notna().sum()
    important_non_null = row[important_cols_check].notna().sum()
    if important_non_null < 8:
        sparse_rows.append((row['Date'], important_non_null, non_null))

if sparse_rows:
    print(f"\n⚠️  Found {len(sparse_rows)} rows with fewer than 8 important non-null values:")
    for date, important_count, total_count in sparse_rows:
        print(f"  {date}: {important_count} important, {total_count} total non-null values")
else:
    print(f"\n✓ All rows have at least 8 important non-null values!")

# Show population growth data alignment check
print(f"\nPopulation Growth Data Check:")
pop_growth_data = master_df_filtered[['Date', 'US_Population_Growth_Rate_FRED']].dropna()
print(f"- Population growth data points: {len(pop_growth_data)}")
if len(pop_growth_data) > 0:
    print(f"- Date range: {pop_growth_data['Date'].min()} to {pop_growth_data['Date'].max()}")
    print(f"- Sample values: {pop_growth_data.head(3)['US_Population_Growth_Rate_FRED'].tolist()}")

## STL Imputation for Master DataFrame


In [ ]:
# Apply STL imputation to fill missing CAD_Softwood_Export_to_US values
print("Applying STL imputation to master dataframe...")

# Check missing values before imputation
before_missing = master_df_filtered['CAD_Softwood_Export_to_US'].isna().sum()
before_total = len(master_df_filtered)
print(f"\nBefore STL imputation:")
print(f"- Total rows: {before_total}")
print(f"- Missing CAD_Softwood_Export_to_US values: {before_missing} ({before_missing/before_total*100:.1f}%)")

# Apply STL imputation
master_df_final = impute_master_df_softwood_stl(master_df_filtered)

# Check for and remove any duplicate rows that may have been created
print("\nChecking for duplicate rows...")
initial_rows = len(master_df_final)
duplicate_mask = master_df_final.duplicated(subset=['Date'], keep='first')
duplicate_count = duplicate_mask.sum()

if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate row(s). Removing duplicates...")
    duplicate_dates = master_df_final[duplicate_mask]['Date'].tolist()
    print(f"Duplicate dates: {duplicate_dates}")
    
    # Keep only the first occurrence of each date
    master_df_final = master_df_final[~duplicate_mask].reset_index(drop=True)
    final_rows = len(master_df_final)
    print(f"Removed {initial_rows - final_rows} duplicate row(s).")
else:
    print("No duplicate rows found.")

# Check missing values after imputation
after_missing = master_df_final['CAD_Softwood_Export_to_US'].isna().sum()
after_total = len(master_df_final)
print(f"\nAfter STL imputation:")
print(f"- Total rows: {after_total}")
print(f"- Missing CAD_Softwood_Export_to_US values: {after_missing} ({after_missing/after_total*100:.1f}%)")

# Verify no missing values remain
if after_missing == 0:
    print("✓ SUCCESS: All missing CAD_Softwood_Export_to_US values have been filled!")
else:
    print(f"⚠️  WARNING: {after_missing} missing values still remain")

# Show summary statistics
print(f"\nImputation Summary:")
print(f"- Values imputed: {before_missing - after_missing}")
print(f"- Imputation success rate: {((before_missing - after_missing) / before_missing * 100) if before_missing > 0 else 100:.1f}%")

# Display sample of the final data
print(f"\nSample of final data with imputed values:")
sample_data = master_df_final[['Date', 'CAD_Softwood_Export_to_US']].head(10)
print(sample_data)


# Modeling Pipeline

## Step 1: Missing Value Analysis and Imputation

Before we can build our predictive models, we need to handle missing values in our feature variables. While the target variable (CAD_Softwood_Export_to_US) has been successfully imputed using STL decomposition, several predictor variables still contain missing values.

This analysis will:
1. Identify which features have missing data
2. Analyze the missingness patterns
3. Determine the best imputation strategy for each variable
4. Implement the chosen imputation method

In [ ]:
# Missing Data Analysis
print("="*70)
print("MISSING DATA ANALYSIS")
print("="*70)

# Get all numeric columns except Date and target
feature_cols = [col for col in master_df_final.columns
                if col not in ['Date', 'CAD_Softwood_Export_to_US']]

print(f"\nAnalyzing {len(feature_cols)} predictor variables...")
print(f"Dataset: {len(master_df_final)} quarterly observations")
print(f"Date range: {master_df_final['Date'].min()} to {master_df_final['Date'].max()}\n")

# Calculate missing data statistics
missing_stats = []
for col in feature_cols:
    total_count = len(master_df_final)
    missing_count = master_df_final[col].isna().sum()
    missing_pct = (missing_count / total_count) * 100
    present_count = total_count - missing_count

    missing_stats.append({
        'Variable': col,
        'Total': total_count,
        'Present': present_count,
        'Missing': missing_count,
        'Missing_Pct': missing_pct
    })

# Create DataFrame and sort by missing percentage
missing_df = pd.DataFrame(missing_stats).sort_values('Missing_Pct', ascending=False)

# Display results
print("Missing Data Summary (sorted by missing percentage):")
print("-" * 70)
for idx, row in missing_df.iterrows():
    if row['Missing'] > 0:
        print(f"{row['Variable']:40s} | {row['Present']:2d}/{row['Total']:2d} | "
              f"Missing: {row['Missing']:2d} ({row['Missing_Pct']:5.1f}%)")

# Count features by missing data category
no_missing = len(missing_df[missing_df['Missing'] == 0])
low_missing = len(missing_df[(missing_df['Missing'] > 0) & (missing_df['Missing_Pct'] <= 10)])
medium_missing = len(missing_df[(missing_df['Missing_Pct'] > 10) & (missing_df['Missing_Pct'] <= 30)])
high_missing = len(missing_df[missing_df['Missing_Pct'] > 30])

print("\n" + "="*70)
print("MISSING DATA CATEGORIES:")
print("="*70)
print(f"Complete features (0% missing):           {no_missing} variables")
print(f"Low missingness (1-10% missing):          {low_missing} variables")
print(f"Medium missingness (10-30% missing):      {medium_missing} variables")
print(f"High missingness (>30% missing):          {high_missing} variables")

# Visualize missing data pattern
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of missing percentages
vars_with_missing = missing_df[missing_df['Missing'] > 0]
if len(vars_with_missing) > 0:
    colors_missing = ['#d7191c' if x > 30 else '#fdae61' if x > 10 else '#fee08b'
                      for x in vars_with_missing['Missing_Pct']]
    ax1.barh(range(len(vars_with_missing)), vars_with_missing['Missing_Pct'],
             color=colors_missing)
    ax1.set_yticks(range(len(vars_with_missing)))
    ax1.set_yticklabels([v.replace('_', ' ') for v in vars_with_missing['Variable']],
                         fontsize=9)
    ax1.set_xlabel('Missing Data (%)', fontsize=11)
    ax1.set_title('Missing Data by Variable', fontsize=12, fontweight='bold')
    ax1.axvline(x=10, color='orange', linestyle='--', alpha=0.5, label='10% threshold')
    ax1.axvline(x=30, color='red', linestyle='--', alpha=0.5, label='30% threshold')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='x')

# Heatmap of missing data pattern over time
# Create binary missing data matrix
missing_matrix = master_df_final[feature_cols].isna().astype(int)
missing_by_var = missing_matrix.sum(axis=0).sort_values(ascending=False)
vars_to_show = missing_by_var[missing_by_var > 0].index.tolist()

if len(vars_to_show) > 0:
    missing_subset = missing_matrix[vars_to_show].T
    sns.heatmap(missing_subset, cmap=['#2c7bb6', '#d7191c'],
                cbar_kws={'label': 'Missing (1) / Present (0)'},
                yticklabels=[v.replace('_', ' ') for v in vars_to_show],
                xticklabels=False, ax=ax2)
    ax2.set_title('Missing Data Pattern Over Time', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Time (Quarterly Observations)', fontsize=11)
    ax2.set_ylabel('Variables', fontsize=11)

plt.tight_layout()
plt.show()

# Return summary for decision-making
print("\n" + "="*70)
print("IMPUTATION STRATEGY RECOMMENDATIONS:")
print("="*70)

for idx, row in missing_df[missing_df['Missing'] > 0].iterrows():
    var_name = row['Variable']
    miss_pct = row['Missing_Pct']

    if miss_pct > 50:
        recommendation = "DROPPING, too much missing data"
    elif miss_pct > 30:
        recommendation = "Forward fill or median imputation"
    elif miss_pct > 10:
        recommendation = "Forward fill (time series) or mean/median imputation"
    else:
        recommendation = "Forward fill or mean imputation (minimal impact)"

    print(f"{var_name:40s} ({miss_pct:5.1f}% missing): {recommendation}")

### 1.1 Missing Value Imputation Strategy

Based on the missing data analysis, we will implement the following strategy:

**Variables to DROP:**
- `CAD_Export_Price_Lumber` (57.0% missing) - Too much missing data to reliably impute

**Variables to IMPUTE using STL Decomposition:**
- `CAD_Building Permits` (26.6% missing, 21 missing quarters)
- `CAD_BP_Single_Housing` (26.6% missing, 21 missing quarters)
- `US_Households_Number` (3.8% missing, 3 missing quarters)

**Rationale:** STL decomposition is appropriate for these time series variables because it:
- Preserves seasonal patterns inherent in quarterly economic data
- Captures trend components (growth/decline over time)
- Handles outliers robustly
- Maintains consistency with our target variable imputation methodology

In [ ]:
# Function to impute missing values using STL decomposition (reusable for any variable)
def impute_variable_stl(df, variable_name, period=4, seasonal=11):
    """
    Impute missing values in a variable using STL decomposition.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with 'Date' column and the variable to impute
    variable_name : str
        Name of the column to impute
    period : int
        Seasonal period (4 for quarterly data)
    seasonal : int
        Seasonal window parameter for STL
        
    Returns:
    --------
    pandas.DataFrame
        Updated dataframe with imputed values
    """
    
    print(f"\n{'='*70}")
    print(f"STL IMPUTATION: {variable_name}")
    print(f"{'='*70}")
    
    # Create a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Extract Date and variable columns
    var_data = df_copy[['Date', variable_name]].copy()
    
    # Convert Date to datetime and sort
    var_data['Date'] = pd.to_datetime(var_data['Date'])
    var_data = var_data.sort_values('Date').reset_index(drop=True)
    
    # Check initial missing values
    initial_missing = var_data[variable_name].isna().sum()
    total_values = len(var_data)
    
    print(f"\nInitial analysis:")
    print(f"  Total data points: {total_values}")
    print(f"  Missing values: {initial_missing} ({initial_missing/total_values*100:.1f}%)")
    print(f"  Date range: {var_data['Date'].min().date()} to {var_data['Date'].max().date()}")
    
    if initial_missing == 0:
        print("  No missing values found. Skipping imputation.")
        return df_copy
    
    # Convert values to numeric
    var_data[variable_name] = pd.to_numeric(var_data[variable_name], errors='coerce')
    
    # Check if we have enough data for STL decomposition
    non_null_count = var_data[variable_name].notna().sum()
    
    print(f"\nSTL Decomposition Setup:")
    print(f"  Non-null values: {non_null_count}")
    print(f"  Data completeness: {non_null_count/total_values*100:.1f}%")
    
    if non_null_count < 8:  # Need at least 2 years of quarterly data
        print("  WARNING: Insufficient data for STL decomposition.")
        print("  Using linear interpolation instead...")
        var_data[variable_name] = var_data[variable_name].interpolate(method='linear')
    else:
        # Prepare data for STL decomposition
        var_ts = var_data.set_index('Date')[variable_name]
        
        # Store original missing mask
        original_missing_mask = var_ts.isna()
        
        # Forward fill and backward fill to handle edge cases for STL
        ts_filled = var_ts.ffill().bfill()
        
        try:
            # Perform STL decomposition with quarterly parameters
            print(f"  Performing STL decomposition...")
            print(f"  Parameters: seasonal={seasonal}, period={period} (quarterly), robust=True")
            
            stl = STL(ts_filled, seasonal=seasonal, period=period, robust=True)
            result = stl.fit()
            
            # Extract components
            trend_component = result.trend
            seasonal_component = result.seasonal
            residual_component = result.resid
            
            print("  STL decomposition completed successfully!")
            print(f"  Trend range: {trend_component.min():.2f} to {trend_component.max():.2f}")
            print(f"  Seasonal range: {seasonal_component.min():.2f} to {seasonal_component.max():.2f}")
            print(f"  Residual std: {residual_component.std():.2f}")
            
            # Impute missing values using trend + seasonal components
            if original_missing_mask.any():
                imputed_values = (trend_component[original_missing_mask] + 
                                 seasonal_component[original_missing_mask])
                var_ts.loc[original_missing_mask] = imputed_values
                
                print(f"\nImputation Results:")
                print(f"  Imputed {original_missing_mask.sum()} missing values")
                print(f"  Imputed value range: {imputed_values.min():.2f} to {imputed_values.max():.2f}")
                
                # Show some examples of imputed values
                imputed_indices = var_ts.index[original_missing_mask]
                if len(imputed_indices) <= 5:
                    print(f"  Imputed dates: {[d.date() for d in imputed_indices]}")
                else:
                    print(f"  Sample imputed dates: {[d.date() for d in imputed_indices[:3]]} ...")
            
            # Update the dataframe
            var_data[variable_name] = var_ts.values
            
        except Exception as e:
            print(f"  STL decomposition failed: {e}")
            print("  Falling back to linear interpolation...")
            var_data[variable_name] = var_data[variable_name].interpolate(method='linear')
    
    # Convert Date back to original format
    var_data['Date'] = var_data['Date'].dt.date
    
    # Create a mapping dictionary for imputed values
    imputed_mapping = dict(zip(var_data['Date'], var_data[variable_name]))
    
    # Apply imputed values to the original dataframe
    df_copy[variable_name] = df_copy['Date'].map(imputed_mapping).fillna(df_copy[variable_name])
    
    # Final verification
    final_missing = df_copy[variable_name].isna().sum()
    print(f"\nFinal Results:")
    print(f"  Missing values after imputation: {final_missing}")
    print(f"  Imputation success: {'✓' if final_missing == 0 else '✗'}")
    
    return df_copy


# Step 1: Drop CAD_Export_Price_Lumber
print("="*70)
print("STEP 1: DROPPING VARIABLES WITH EXCESSIVE MISSING DATA")
print("="*70)

print(f"\nBefore dropping:")
print(f"  Total features: {len(master_df_final.columns) - 1}")  # Exclude Date
print(f"  CAD_Export_Price_Lumber missing: {master_df_final['CAD_Export_Price_Lumber'].isna().sum()}/79 (57.0%)")

# Create a copy and drop the variable
df_imputed = master_df_final.drop(columns=['CAD_Export_Price_Lumber']).copy()

print(f"\nAfter dropping:")
print(f"  Total features: {len(df_imputed.columns) - 1}")  # Exclude Date
print(f"  Dropped: CAD_Export_Price_Lumber")


# Step 2: Impute using STL decomposition
print("\n" + "="*70)
print("STEP 2: STL DECOMPOSITION IMPUTATION")
print("="*70)

# Impute CAD_Building Permits
df_imputed = impute_variable_stl(df_imputed, 'CAD_Building Permits', period=4, seasonal=11)

# Impute CAD_BP_Single_Housing
df_imputed = impute_variable_stl(df_imputed, 'CAD_BP_Single_Housing', period=4, seasonal=11)

# Impute US_Households_Number
df_imputed = impute_variable_stl(df_imputed, 'US_Households_Number', period=4, seasonal=11)


# Step 3: Verification and Summary
print("\n" + "="*70)
print("IMPUTATION SUMMARY")
print("="*70)

# Check for any remaining missing values
remaining_missing = df_imputed.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]

if len(remaining_missing) == 0:
    print("\n✓ SUCCESS: All missing values have been imputed!")
    print(f"  Dataset is now 100% complete with {len(df_imputed)} observations")
else:
    print(f"\n⚠️  WARNING: {len(remaining_missing)} variables still have missing values:")
    for var, count in remaining_missing.items():
        print(f"  {var}: {count} missing")

# Show summary statistics
print(f"\nFinal Dataset Summary:")
print(f"  Total rows: {len(df_imputed)}")
print(f"  Total columns: {len(df_imputed.columns)} (1 Date + {len(df_imputed.columns)-1} features)")
print(f"  Date range: {df_imputed['Date'].min()} to {df_imputed['Date'].max()}")
print(f"  Overall completeness: {(1 - df_imputed.isnull().sum().sum() / (len(df_imputed) * len(df_imputed.columns))) * 100:.1f}%")

# Display sample of imputed data
print(f"\nSample of imputed dataset (first 5 rows):")
print(df_imputed[['Date', 'CAD_Building Permits', 'CAD_BP_Single_Housing', 'US_Households_Number']].head())

## Summary

In [ ]:
print(f"\n{'='*60}")
print(f"Master DataFrame Summary:")
print(f"{'='*60}")
print(f"Original total rows: {len(master_df)}")
print(f"Filtered total rows: {len(master_df_filtered)}")
print(f"Final total rows (after STL imputation): {len(master_df_final)}")
print(f"Total columns: {len(master_df_final.columns)} (Date + {len(master_df_final.columns)-1} variables)")
print(f"Date range: {master_df_final['Date'].min()} to {master_df_final['Date'].max()}")
print(f"\nColumns: {', '.join(master_df_final.columns.tolist())}")

# Show softwood data completeness
softwood_missing = master_df_final['CAD_Softwood_Export_to_US'].isna().sum()
print(f"\nCAD_Softwood_Export_to_US completeness: {((len(master_df_final) - softwood_missing) / len(master_df_final) * 100):.1f}% ({softwood_missing} missing values)")

In [ ]:
master_df_final.head()

In [ ]:
master_df_final.info()

In [ ]:
master_df_final.describe()

### Save Output

In [ ]:
output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Save to CSV
output_file = os.path.join(output_dir, 'bloomberg_master_dataframe.csv')
master_df_final.to_csv(output_file, index=False)
print(f"Master dataframe saved to: {output_file}")

In [ ]:
# Save to Excel
output_excel = os.path.join(output_dir, 'bloomberg_master_dataframe.xlsx')
master_df_final.to_excel(output_excel, index=False)
print(f"Master dataframe saved to: {output_excel}")

# Exploratory Data Analysis

Setting plotting style

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

## 1. Time Series Overview

### 1.1 Target Variable: Canadian Softwood Exports to US

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(master_df_final['Date'], master_df_final['CAD_Softwood_Export_to_US'], 
        linewidth=2, color='#2c7bb6')
ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Quarterly Exports (Thousand Cubic Meters)', fontsize=11)
ax.set_title('Canada-US Softwood Lumber Exports Over Time', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print(f"Export Statistics:")
print(f"  Mean: {master_df_final['CAD_Softwood_Export_to_US'].mean():,.0f} FBM")
print(f"  Std Dev: {master_df_final['CAD_Softwood_Export_to_US'].std():,.0f} FBM")
print(f"  Min: {master_df_final['CAD_Softwood_Export_to_US'].min():,.0f} FBM")
print(f"  Max: {master_df_final['CAD_Softwood_Export_to_US'].max():,.0f} FBM")

## 2. Key Economic Indicators

### 2.1 US Housing Market Indicators

In [ ]:
housing_vars = ['US_Building_Permits', 'US_Housing_Start', 'US_New_Home_Sales', 
                'US_Existing_Home _Sales']
available_housing = [v for v in housing_vars if v in master_df_final.columns]

if available_housing:
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    axes = axes.flatten()
    
    for i, var in enumerate(available_housing):
        axes[i].plot(master_df_final['Date'], master_df_final[var], 
                    linewidth=2, color='#d7191c')
        axes[i].set_xlabel('Date', fontsize=10)
        axes[i].set_ylabel('Units', fontsize=10)
        axes[i].set_title(var.replace('_', ' '), fontsize=11, fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 2.2 Price Indicators

In [ ]:
price_vars = ['US_CPI', 'CAD_CPI', 'CAD_Export_Price_Lumber']
available_prices = [v for v in price_vars if v in master_df_final.columns]

if len(available_prices) >= 2:
    fig, axes = plt.subplots(1, len(available_prices), figsize=(14, 5))
    if len(available_prices) == 1:
        axes = [axes]  # Make it iterable for single subplot
    
    for i, var in enumerate(available_prices):
        # Convert to numeric, handling any non-numeric values
        price_data = pd.to_numeric(master_df_final[var], errors='coerce')
        
        axes[i].plot(master_df_final['Date'], price_data, 
                    linewidth=2, color='#fdae61')
        axes[i].set_xlabel('Date', fontsize=10)
        axes[i].set_ylabel('Index/Price', fontsize=10)
        axes[i].set_title(var.replace('_', ' '), fontsize=11, fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics for price indicators
    print("\nPrice Indicators Summary:")
    for var in available_prices:
        price_data = pd.to_numeric(master_df_final[var], errors='coerce')
        non_null_count = price_data.notna().sum()
        print(f"  {var}: {non_null_count} data points")
        if non_null_count > 0:
            print(f"    Range: {price_data.min():.2f} to {price_data.max():.2f}")
else:
    print("Insufficient price indicator data available for plotting")

## 3. Correlation Analysis

### 3.1 Correlation with Target Variable

In [ ]:
# Convert object columns to numeric (excluding Date)
print("Converting columns to numeric for correlation analysis...")
target = 'CAD_Softwood_Export_to_US'

# Get all columns except Date
all_cols = [col for col in master_df_final.columns if col != 'Date']

# Convert each column to numeric, handling errors
for col in all_cols:
    if col != target:  # Target is already numeric
        master_df_final[col] = pd.to_numeric(master_df_final[col], errors='coerce')

# Now select numeric columns (exclude Date)
numeric_cols = master_df_final.select_dtypes(include=[np.number]).columns.tolist()
print(f"Found {len(numeric_cols)} numeric columns: {numeric_cols}")

# Show data completeness after conversion
print(f"\nData completeness after numeric conversion:")
for col in numeric_cols:
    non_null_count = master_df_final[col].notna().sum()
    total_count = len(master_df_final)
    print(f"  {col}: {non_null_count}/{total_count} ({non_null_count/total_count*100:.1f}%)")

In [ ]:
# Calculate correlations with target
print("Calculating correlations with target variable...")
correlations = []
for col in numeric_cols:
    if col != target:
        # Remove rows with missing values in either column
        valid_data = master_df_final[[target, col]].dropna()
        if len(valid_data) > 10:  # Need sufficient data points
            try:
                corr, pval = pearsonr(valid_data[target], valid_data[col])
                correlations.append({
                    'Variable': col,
                    'Correlation': corr,
                    'P-value': pval,
                    'Significant': pval < 0.05,
                    'Data_Points': len(valid_data)
                })
                print(f"  {col}: r={corr:.3f}, p={pval:.3f}, n={len(valid_data)}")
            except Exception as e:
                print(f"  {col}: Error calculating correlation - {e}")

print(f"\nFound {len(correlations)} valid correlations")

if len(correlations) > 0:
    corr_df = pd.DataFrame(correlations).sort_values('Correlation', 
                                                      key=abs, 
                                                      ascending=False)
    print(f"Correlation analysis completed successfully!")
else:
    print("No valid correlations found. Creating empty DataFrame.")
    corr_df = pd.DataFrame(columns=['Variable', 'Correlation', 'P-value', 'Significant', 'Data_Points'])

### 3.2 Top Correlations

In [ ]:
# Plot top correlations
if len(corr_df) > 0:
    top_n = min(10, len(corr_df))  # Use available correlations or 10, whichever is smaller
    top_corr = corr_df.head(top_n)

    fig, ax = plt.subplots(figsize=(10, max(6, len(top_corr) * 0.4)))
    colors = ['#2c7bb6' if x > 0 else '#d7191c' for x in top_corr['Correlation']]
    bars = ax.barh(range(len(top_corr)), top_corr['Correlation'], color=colors)
    ax.set_yticks(range(len(top_corr)))
    ax.set_yticklabels([v.replace('_', ' ') for v in top_corr['Variable']], fontsize=9)
    ax.set_xlabel('Correlation Coefficient', fontsize=11)
    ax.set_title(f'Top {top_n} Correlations with Canadian Softwood Exports', 
                 fontsize=13, fontweight='bold')
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add correlation values as text on bars
    for i, (idx, row) in enumerate(top_corr.iterrows()):
        ax.text(row['Correlation'] + (0.01 if row['Correlation'] > 0 else -0.01), 
                i, f'{row["Correlation"]:.3f}', 
                va='center', ha='left' if row['Correlation'] > 0 else 'right', 
                fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

    print(f"\nTop {top_n} Correlations (with p-values and significance):")
    display_cols = ['Variable', 'Correlation', 'P-value', 'Significant', 'Data_Points']
    print(corr_df[display_cols].head(top_n).to_string(index=False, float_format='%.3f'))
    
    # Summary statistics
    significant_corr = corr_df[corr_df['Significant'] == True]
    print(f"\nCorrelation Summary:")
    print(f"  Total correlations calculated: {len(corr_df)}")
    print(f"  Significant correlations (p < 0.05): {len(significant_corr)}")
    if len(significant_corr) > 0:
        print(f"  Strongest positive correlation: {significant_corr[significant_corr['Correlation'] > 0]['Correlation'].max():.3f}")
        print(f"  Strongest negative correlation: {significant_corr[significant_corr['Correlation'] < 0]['Correlation'].min():.3f}")
else:
    print("No correlations available to plot.")
    print("This might be due to insufficient data or all variables being constant.")


### 3.3 Correlation Matrix Scatter Plots

In [ ]:
# Select key variables for correlation analysis
key_vars = ['CAD_Softwood_Export_to_US', 'US_Housing_Start', 'US_Building_Permits', 
            'US_CPI', 'CAD_Export_Price_Lumber', 'USCAD_Exchange_Rate']

# Filter to variables that exist in the dataset
available_vars = [var for var in key_vars if var in master_df_final.columns]
n_vars = len(available_vars)

# Create correlation matrix plot
fig, axes = plt.subplots(n_vars, n_vars, figsize=(12, 10))
fig.suptitle('Variable Relationships Matrix', fontsize=16, y=0.95)

for i in range(n_vars):
    for j in range(n_vars):
        ax = axes[i, j]
        
        if i == j:
            # Diagonal: show variable name
            ax.text(0.5, 0.5, available_vars[i].replace('_', ' '), 
                   ha='center', va='center', transform=ax.transAxes, fontweight='bold')
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            # Off-diagonal: scatter plot
            x_var = available_vars[j]
            y_var = available_vars[i]
            
            # Get data
            data = master_df_final[[x_var, y_var]].dropna()
            
            if len(data) > 5:
                # Create scatter plot
                ax.scatter(data[x_var], data[y_var], alpha=0.6, s=30)
                
                # Add correlation coefficient
                if len(data) > 10:
                    corr, p_val = pearsonr(data[x_var], data[y_var])
                    ax.text(0.05, 0.95, f'r = {corr:.3f}', 
                           transform=ax.transAxes, fontsize=8, 
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            
            # Set labels
            if i == n_vars - 1:  # Bottom row
                ax.set_xlabel(x_var.replace('_', ' '), fontsize=8)
            if j == 0:  # Left column
                ax.set_ylabel(y_var.replace('_', ' '), fontsize=8)
            
            ax.tick_params(labelsize=6)
            ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Data Completeness

### 4.1 Data Coverage Summary

### 5.1 Scatter Plot: Key Relationships

"Key relationships" refers to the strongest correlations between economic indicators and Canadian softwood exports. These help identify which factors most influence export volumes and can guide forecasting models

In [ ]:
# Get top 3 most correlated variables
if len(corr_df) >= 3:
    top_vars = corr_df.head(3)['Variable'].tolist()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, var in enumerate(top_vars):
        data = master_df_final[[target, var]].dropna()
        
        axes[i].scatter(data[var], data[target], alpha=0.6)
        
        # Add trend line
        z = np.polyfit(data[var], data[target], 1)
        p = np.poly1d(z)
        axes[i].plot(data[var], p(data[var]), "r--", alpha=0.8)
        
        axes[i].set_xlabel(var.replace('_', ' '))
        axes[i].set_ylabel('Softwood Exports')
        corr_val = corr_df[corr_df['Variable'] == var]['Correlation'].values[0]
        axes[i].set_title(f'{var.replace("_", " ")}\n(r = {corr_val:.3f})')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## Step 2: Train/Validation/Test Split (60/20/20)

For time series data, we must use **chronological splitting** to prevent data leakage. Random shuffling would violate the temporal dependency structure and lead to overly optimistic performance estimates.

### Splitting Strategy:
- **Training Set (60%)**: Used to train models - oldest data
- **Validation Set (20%)**: Used for hyperparameter tuning and model selection - middle period
- **Test Set (20%)**: Final evaluation only - most recent data

### Why Chronological Split?
1. **Preserves temporal ordering**: Models are trained on past data to predict future
2. **Realistic evaluation**: Mimics real-world forecasting scenario
3. **Prevents data leakage**: Future information never informs past predictions
4. **Respects autocorrelation**: Time series dependencies remain intact

In [ ]:
# Train/Validation/Test Split (60/20/20 Chronological)
print("="*70)
print("TRAIN/VALIDATION/TEST SPLIT (60/20/20 CHRONOLOGICAL)")
print("="*70)

# Sort by date to ensure chronological order (ascending - oldest to newest)
df_imputed_sorted = df_imputed.sort_values('Date').reset_index(drop=True)

print(f"\nDataset Information:")
print(f"  Total observations: {len(df_imputed_sorted)}")
print(f"  Date range: {df_imputed_sorted['Date'].min()} to {df_imputed_sorted['Date'].max()}")
print(f"  Total features: {len(df_imputed_sorted.columns) - 2}")  # Exclude Date and target

# Calculate split indices
n_total = len(df_imputed_sorted)
n_train = int(n_total * 0.60)
n_val = int(n_total * 0.20)
n_test = n_total - n_train - n_val  # Ensure all observations are used

print(f"\nSplit Sizes:")
print(f"  Training:   {n_train} observations ({n_train/n_total*100:.1f}%)")
print(f"  Validation: {n_val} observations ({n_val/n_total*100:.1f}%)")
print(f"  Test:       {n_test} observations ({n_test/n_total*100:.1f}%)")

# Perform chronological split
train_df = df_imputed_sorted.iloc[:n_train].copy()
val_df = df_imputed_sorted.iloc[n_train:n_train+n_val].copy()
test_df = df_imputed_sorted.iloc[n_train+n_val:].copy()

# Display date ranges for each split
print(f"\nDate Ranges:")
print(f"  Training:   {train_df['Date'].min()} to {train_df['Date'].max()}")
print(f"  Validation: {val_df['Date'].min()} to {val_df['Date'].max()}")
print(f"  Test:       {test_df['Date'].min()} to {test_df['Date'].max()}")

# Separate features (X) and target (y) for each split
target_col = 'CAD_Softwood_Export_to_US'
feature_cols = [col for col in df_imputed_sorted.columns if col not in ['Date', target_col]]

X_train = train_df[feature_cols].copy()
y_train = train_df[target_col].copy()

X_val = val_df[feature_cols].copy()
y_val = val_df[target_col].copy()

X_test = test_df[feature_cols].copy()
y_test = test_df[target_col].copy()

print(f"\nFeature Matrix Shapes:")
print(f"  X_train: {X_train.shape} (rows × features)")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")

print(f"\nTarget Vector Shapes:")
print(f"  y_train: {y_train.shape}")
print(f"  y_val:   {y_val.shape}")
print(f"  y_test:  {y_test.shape}")

# Verify no data leakage - dates should not overlap
print(f"\n{'='*70}")
print("DATA LEAKAGE CHECK")
print(f"{'='*70}")

train_end = train_df['Date'].max()
val_start = val_df['Date'].min()
val_end = val_df['Date'].max()
test_start = test_df['Date'].min()

if train_end < val_start and val_end < test_start:
    print("✓ PASS: No temporal overlap between splits")
    print(f"  Training ends:     {train_end}")
    print(f"  Validation starts: {val_start}")
    print(f"  Validation ends:   {val_end}")
    print(f"  Test starts:       {test_start}")
else:
    print("✗ WARNING: Temporal overlap detected!")
    print(f"  Training ends:     {train_end}")
    print(f"  Validation starts: {val_start}")
    print(f"  Validation ends:   {val_end}")
    print(f"  Test starts:       {test_start}")

# Summary statistics for target variable across splits
print(f"\n{'='*70}")
print("TARGET VARIABLE DISTRIBUTION ACROSS SPLITS")
print(f"{'='*70}")

print(f"\nTraining Set:")
print(f"  Mean:   {y_train.mean():,.0f}")
print(f"  Std:    {y_train.std():,.0f}")
print(f"  Min:    {y_train.min():,.0f}")
print(f"  Max:    {y_train.max():,.0f}")

print(f"\nValidation Set:")
print(f"  Mean:   {y_val.mean():,.0f}")
print(f"  Std:    {y_val.std():,.0f}")
print(f"  Min:    {y_val.min():,.0f}")
print(f"  Max:    {y_val.max():,.0f}")

print(f"\nTest Set:")
print(f"  Mean:   {y_test.mean():,.0f}")
print(f"  Std:    {y_test.std():,.0f}")
print(f"  Min:    {y_test.min():,.0f}")
print(f"  Max:    {y_test.max():,.0f}")

# Visualize the split
fig, ax = plt.subplots(figsize=(14, 5))

# Plot all data
ax.plot(train_df['Date'], y_train, 'o-', label='Training', color='#2c7bb6', linewidth=2, markersize=5)
ax.plot(val_df['Date'], y_val, 'o-', label='Validation', color='#fdae61', linewidth=2, markersize=5)
ax.plot(test_df['Date'], y_test, 'o-', label='Test', color='#d7191c', linewidth=2, markersize=5)

# Add vertical lines to separate splits
ax.axvline(x=train_df['Date'].max(), color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax.axvline(x=val_df['Date'].max(), color='gray', linestyle='--', alpha=0.5, linewidth=1)

ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Softwood Exports (Thousand Cubic Meters)', fontsize=11)
ax.set_title('Train/Validation/Test Split (Chronological)', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n{'='*70}")
print("SPLIT COMPLETE - READY FOR FEATURE ENGINEERING")
print(f"{'='*70}")

## Step 3: Feature Engineering

We will create features that capture:
1. **Temporal patterns**: Trend and seasonality
2. **Historical dependencies**: Lag features
3. **Domain knowledge**: Economic relationships and interactions

### Feature Engineering Strategy:

**3.1 Time-Based Features:**
- Linear trend (captures long-term growth/decline)
- Quarterly dummy variables (captures seasonality)

**3.2 Lag Features:**
- Target lags (past export values predict future)
- Predictor lags (past economic conditions predict future exports)

**3.3 Derived Features:**
- Moving averages (smoothed trends)
- Year-over-year growth rates (economic dynamics)
- Interaction terms (multiplicative effects)

Features must be engineered on the **training set first**, then applied to validation and test sets to prevent data leakage.

In [ ]:
# Step 3.1: Time-Based Features (Trend + Seasonality)
print("="*70)
print("STEP 3.1: TIME-BASED FEATURE ENGINEERING")
print("="*70)

# We need to work with the full sorted dataframe to maintain temporal order
# Then split again after feature engineering

# Start with the sorted dataframe from Step 2
df_features = df_imputed_sorted.copy()

print(f"\nStarting dataset: {df_features.shape}")

# Feature 1: Linear Trend (0, 1, 2, ..., n-1)
# Captures long-term growth or decline in exports
df_features['trend'] = np.arange(len(df_features))

print(f"\n1. TREND VARIABLE:")
print(f"   Created linear trend variable")
print(f"   Range: {df_features['trend'].min()} to {df_features['trend'].max()}")
print(f"   Interpretation: Captures systematic increase/decrease over time")

# Feature 2: Quarterly Dummy Variables
# Extract quarter from date
df_features['Date_dt'] = pd.to_datetime(df_features['Date'])
df_features['quarter'] = df_features['Date_dt'].dt.quarter

# Create dummy variables (drop Q1 to avoid multicollinearity)
quarter_dummies = pd.get_dummies(df_features['quarter'], prefix='Q', drop_first=True)
df_features = pd.concat([df_features, quarter_dummies], axis=1)

print(f"\n2. QUARTERLY SEASONALITY:")
print(f"   Created dummy variables: Q_2, Q_3, Q_4 (Q1 is baseline)")
print(f"   Interpretation:")
print(f"   - Q_2 = 1 if Q2, 0 otherwise (Spring)")
print(f"   - Q_3 = 1 if Q3, 0 otherwise (Summer)")
print(f"   - Q_4 = 1 if Q4, 0 otherwise (Fall)")
print(f"   - Q1 (Winter) is captured in the intercept")

# Drop temporary columns
df_features = df_features.drop(['Date_dt', 'quarter'], axis=1)

# Summary
new_features = ['trend', 'Q_2', 'Q_3', 'Q_4']
print(f"\n{'='*70}")
print(f"TIME-BASED FEATURES CREATED: {len(new_features)}")
print(f"{'='*70}")
for feat in new_features:
    print(f"  - {feat}")

print(f"\nUpdated dataset shape: {df_features.shape}")
print(f"Total features added: {len(new_features)}")

### 3.2 Lag Features

Lag features capture **temporal dependencies** - the idea that past values help predict future values.

**Why Lags Are Important:**
- **Target lags**: Past export volumes influence future exports (autocorrelation)
- **Predictor lags**: Economic indicators take time to affect exports (e.g., housing permits → construction → lumber demand)

**Strategy:**
- **Target variable lags**: lag-1 (previous quarter), lag-4 (same quarter last year)
- **Key predictor lags**: lag-1 for top correlated housing variables
- **Trade-off**: More lags = more features but fewer observations (due to NaN values)

**Note**: Lag features create missing values at the beginning of the time series, which we'll handle by dropping those rows.

In [ ]:
# Step 3.2: Lag Features
print("="*70)
print("STEP 3.2: LAG FEATURE ENGINEERING")
print("="*70)

print(f"\nStarting dataset: {df_features.shape}")
print(f"Date range: {df_features['Date'].min()} to {df_features['Date'].max()}")

# List to track new lag features
lag_features_created = []

# Create lag features for TARGET variable (CAD_Softwood_Export_to_US)
print(f"\n1. TARGET VARIABLE LAGS:")

target_col = 'CAD_Softwood_Export_to_US'

# Lag-1: Previous quarter's exports
df_features[f'{target_col}_lag1'] = df_features[target_col].shift(1)
lag_features_created.append(f'{target_col}_lag1')
print(f"   ✓ Created {target_col}_lag1 (previous quarter)")

# Lag-2: Two quarters ago
df_features[f'{target_col}_lag2'] = df_features[target_col].shift(2)
lag_features_created.append(f'{target_col}_lag2')
print(f"   ✓ Created {target_col}_lag2 (two quarters ago)")

# Lag-4: Same quarter last year (captures yearly seasonality)
df_features[f'{target_col}_lag4'] = df_features[target_col].shift(4)
lag_features_created.append(f'{target_col}_lag4')
print(f"   ✓ Created {target_col}_lag4 (same quarter last year)")

# Create lag features for KEY PREDICTORS
# Based on correlation analysis, these had the strongest relationships with target:
# - US_BP_Single_Housing (r=0.752)
# - US_New_Home_Sales (r=0.744)
# - US_Housing_Start (r=0.732)

print(f"\n2. KEY PREDICTOR LAGS (lag-1 only to limit feature growth):")

key_predictors = [
    'US _BP_Single_Housing',
    'US_New_Home_Sales', 
    'US_Housing_Start',
    'US_Building_Permits'
]

for pred in key_predictors:
    if pred in df_features.columns:
        lag_col = f'{pred}_lag1'
        df_features[lag_col] = df_features[pred].shift(1)
        lag_features_created.append(lag_col)
        print(f"   ✓ Created {lag_col}")

# Check for missing values created by lagging
print(f"\n3. MISSING VALUES ANALYSIS:")
print(f"   Lag features create NaN values at the beginning of the time series")

# Count NaN values in lag features
for lag_feat in lag_features_created:
    nan_count = df_features[lag_feat].isna().sum()
    print(f"   {lag_feat}: {nan_count} NaN values")

# Drop rows with ANY missing values in lag features
# This is necessary because we can't make predictions without complete feature sets
print(f"\n4. HANDLING MISSING VALUES:")
print(f"   Before dropping: {len(df_features)} rows")

# Store original date range
original_start = df_features['Date'].min()
original_end = df_features['Date'].max()

# Drop rows with NaN in lag features
df_features_clean = df_features.dropna(subset=lag_features_created).reset_index(drop=True)

print(f"   After dropping:  {len(df_features_clean)} rows")
print(f"   Rows lost:       {len(df_features) - len(df_features_clean)}")
print(f"   Original date range: {original_start} to {original_end}")
print(f"   New date range:      {df_features_clean['Date'].min()} to {df_features_clean['Date'].max()}")

# Update the main dataframe
df_features = df_features_clean.copy()

# Summary
print(f"\n{'='*70}")
print(f"LAG FEATURES CREATED: {len(lag_features_created)}")
print(f"{'='*70}")
for feat in lag_features_created:
    print(f"  - {feat}")

print(f"\nUpdated dataset shape: {df_features.shape}")
print(f"Total lag features added: {len(lag_features_created)}")
print(f"\nNote: We lost the first few observations due to lagging, but this is")
print(f"      necessary to avoid using future information in our predictions.")

### 3.3 Derived Features

Derived features combine existing variables to create new insights based on **domain knowledge** about economics and housing markets.

**Strategy:**
1. **Moving Averages**: Smooth short-term fluctuations to reveal trends
2. **Year-over-Year Growth Rates**: Capture economic momentum (expansion/contraction)
3. **Interaction Terms**: Capture multiplicative effects between variables

**Rationale:**
- **Moving averages** reduce noise in housing indicators
- **Growth rates** are often better predictors than absolute levels for economic data
- **Interactions** capture compound effects (e.g., high mortgage rates × many permits)

**Constraint**: We limit derived features to avoid overfitting given our sample size (~75 observations).

In [ ]:
# Step 3.3: Derived Features
print("="*70)
print("STEP 3.3: DERIVED FEATURE ENGINEERING")
print("="*70)

print(f"\nStarting dataset: {df_features.shape}")

# List to track new derived features
derived_features_created = []

# 1. MOVING AVERAGES (2-quarter rolling average for key housing indicators)
print(f"\n1. MOVING AVERAGES (2-quarter rolling window):")
print(f"   Purpose: Smooth short-term fluctuations, reduce noise")

ma_variables = [
    'US_Housing_Start',
    'US_Building_Permits',
    'US_New_Home_Sales'
]

for var in ma_variables:
    if var in df_features.columns:
        ma_col = f'{var}_MA2'
        df_features[ma_col] = df_features[var].rolling(window=2, min_periods=1).mean()
        derived_features_created.append(ma_col)
        print(f"   ✓ Created {ma_col}")


# 2. YEAR-OVER-YEAR GROWTH RATES (4-quarter change)
print(f"\n2. YEAR-OVER-YEAR GROWTH RATES:")
print(f"   Purpose: Capture economic momentum and change direction")

# For target variable
target_col = 'CAD_Softwood_Export_to_US'
yoy_target = f'{target_col}_YoY_growth'
df_features[yoy_target] = df_features[target_col].pct_change(periods=4) * 100
derived_features_created.append(yoy_target)
print(f"   ✓ Created {yoy_target} (% change from same quarter last year)")

# For key economic indicators
yoy_variables = ['US_GDP', 'CAD_GDP']

for var in yoy_variables:
    if var in df_features.columns:
        yoy_col = f'{var}_YoY_growth'
        df_features[yoy_col] = df_features[var].pct_change(periods=4) * 100
        derived_features_created.append(yoy_col)
        print(f"   ✓ Created {yoy_col}")


# 3. INTERACTION TERMS (multiplicative effects)
print(f"\n3. INTERACTION TERMS:")
print(f"   Purpose: Capture compound effects between related variables")

# Interaction 1: Housing starts × Mortgage rates
# Rationale: Impact of housing activity depends on financing costs
if 'US_Housing_Start' in df_features.columns and 'US_Mortgage_Interest_30Y' in df_features.columns:
    interaction1 = 'US_Housing_Start_x_Mortgage_Rate'
    df_features[interaction1] = df_features['US_Housing_Start'] * df_features['US_Mortgage_Interest_30Y']
    derived_features_created.append(interaction1)
    print(f"   ✓ Created {interaction1}")
    print(f"      Rationale: Housing activity impact varies with financing costs")

# Interaction 2: Exchange rate × Housing activity
# Rationale: Canadian exports to US depend on both demand (housing) and price competitiveness (exchange rate)
if 'USCAD_Exchange_Rate' in df_features.columns and 'US_Building_Permits' in df_features.columns:
    interaction2 = 'Exchange_Rate_x_Building_Permits'
    df_features[interaction2] = df_features['USCAD_Exchange_Rate'] * df_features['US_Building_Permits']
    derived_features_created.append(interaction2)
    print(f"   ✓ Created {interaction2}")
    print(f"      Rationale: Export competitiveness depends on both price and demand")


# 4. HANDLE MISSING VALUES FROM DERIVED FEATURES
print(f"\n4. HANDLING MISSING VALUES FROM DERIVED FEATURES:")

# Count NaN values in derived features
nan_counts = {}
for feat in derived_features_created:
    nan_count = df_features[feat].isna().sum()
    if nan_count > 0:
        nan_counts[feat] = nan_count

if len(nan_counts) > 0:
    print(f"   Missing values detected in derived features:")
    for feat, count in nan_counts.items():
        print(f"   - {feat}: {count} NaN values")
    
    print(f"\n   Before dropping: {len(df_features)} rows")
    df_features_clean = df_features.dropna().reset_index(drop=True)
    print(f"   After dropping:  {len(df_features_clean)} rows")
    print(f"   Rows lost:       {len(df_features) - len(df_features_clean)}")
    
    df_features = df_features_clean.copy()
else:
    print(f"   No missing values detected in derived features")


# Summary
print(f"\n{'='*70}")
print(f"DERIVED FEATURES CREATED: {len(derived_features_created)}")
print(f"{'='*70}")
for feat in derived_features_created:
    print(f"  - {feat}")

print(f"\nFinal dataset shape: {df_features.shape}")
print(f"Total derived features added: {len(derived_features_created)}")

# Overall feature engineering summary
print(f"\n{'='*70}")
print(f"FEATURE ENGINEERING COMPLETE - SUMMARY")
print(f"{'='*70}")

# Count all engineered features
time_based = ['trend', 'Q_2', 'Q_3', 'Q_4']
print(f"\nFeature Categories:")
print(f"  Time-based features:  {len(time_based)}")
print(f"  Lag features:         7")
print(f"  Derived features:     {len(derived_features_created)}")
print(f"  Total new features:   {len(time_based) + 7 + len(derived_features_created)}")

print(f"\nFinal Dataset:")
print(f"  Total rows:    {len(df_features)}")
print(f"  Total columns: {len(df_features.columns)}")
print(f"  Date range:    {df_features['Date'].min()} to {df_features['Date'].max()}")

# Show sample of engineered features
print(f"\nSample of engineered dataset (first 3 rows):")
sample_cols = ['Date', target_col, 'trend', 'Q_2', f'{target_col}_lag1', 
               'US_Housing_Start_MA2', f'{target_col}_YoY_growth']
available_sample_cols = [col for col in sample_cols if col in df_features.columns]
print(df_features[available_sample_cols].head(3))

## Step 4: Re-split Data and Apply Feature Scaling

After feature engineering, we need to:
1. **Re-split** the data (since we lost observations from lagging/YoY growth)
2. **Scale features** using StandardScaler (required for Ridge, Lasso, ElasticNet)

### Why Re-split?
- Feature engineering reduced our dataset from 79 → ~71 observations
- Need to maintain 60/20/20 split on the final cleaned dataset

### Why Feature Scaling?
- **Regularized models** (Ridge, Lasso, ElasticNet) are sensitive to feature scale
- Variables with larger ranges dominate the penalty term
- StandardScaler: transforms features to mean=0, std=1
- **Important**: Fit scaler on training set only, then transform val/test (prevent data leakage)

In [ ]:
# Step 4: Re-split Data and Apply Feature Scaling
print("="*70)
print("STEP 4: RE-SPLIT DATA AND APPLY FEATURE SCALING")
print("="*70)

# 1. RE-SPLIT DATA (60/20/20 chronological)
print(f"\n1. RE-SPLITTING DATA AFTER FEATURE ENGINEERING:")
print(f"   Dataset is already sorted chronologically from feature engineering")

print(f"\nFinal dataset for modeling:")
print(f"  Total rows: {len(df_features)}")
print(f"  Date range: {df_features['Date'].min()} to {df_features['Date'].max()}")

# Calculate new split indices
n_total = len(df_features)
n_train = int(n_total * 0.60)
n_val = int(n_total * 0.20)
n_test = n_total - n_train - n_val

print(f"\nNew split sizes:")
print(f"  Training:   {n_train} observations ({n_train/n_total*100:.1f}%)")
print(f"  Validation: {n_val} observations ({n_val/n_total*100:.1f}%)")
print(f"  Test:       {n_test} observations ({n_test/n_total*100:.1f}%)")

# Perform split
train_df_final = df_features.iloc[:n_train].copy()
val_df_final = df_features.iloc[n_train:n_train+n_val].copy()
test_df_final = df_features.iloc[n_train+n_val:].copy()

print(f"\nDate ranges:")
print(f"  Training:   {train_df_final['Date'].min()} to {train_df_final['Date'].max()}")
print(f"  Validation: {val_df_final['Date'].min()} to {val_df_final['Date'].max()}")
print(f"  Test:       {test_df_final['Date'].min()} to {test_df_final['Date'].max()}")

# Separate features and target
target_col = 'CAD_Softwood_Export_to_US'
feature_cols_final = [col for col in df_features.columns if col not in ['Date', target_col]]

X_train_final = train_df_final[feature_cols_final].copy()
y_train_final = train_df_final[target_col].copy()

X_val_final = val_df_final[feature_cols_final].copy()
y_val_final = val_df_final[target_col].copy()

X_test_final = test_df_final[feature_cols_final].copy()
y_test_final = test_df_final[target_col].copy()

print(f"\nFeature matrix shapes:")
print(f"  X_train: {X_train_final.shape}")
print(f"  X_val:   {X_val_final.shape}")
print(f"  X_test:  {X_test_final.shape}")


# DATA QUALITY CHECK BEFORE SCALING
print(f"\n{'='*70}")
print("DATA QUALITY CHECK BEFORE SCALING")
print("="*70)

# First, ensure all columns are numeric
print("\n1. Converting all features to numeric:")
non_numeric_cols = X_train_final.select_dtypes(exclude=[np.number]).columns.tolist()
if len(non_numeric_cols) > 0:
    print(f"   Found {len(non_numeric_cols)} non-numeric columns:")
    for col in non_numeric_cols:
        print(f"   - {col}: {X_train_final[col].dtype}")
        # Convert to numeric, coercing errors to NaN
        X_train_final[col] = pd.to_numeric(X_train_final[col], errors='coerce')
        X_val_final[col] = pd.to_numeric(X_val_final[col], errors='coerce')
        X_test_final[col] = pd.to_numeric(X_test_final[col], errors='coerce')
    print("   ✓ Converted all columns to numeric")
else:
    print("   ✓ All columns are already numeric")

# Check for inf and NaN values in features
print("\n2. Checking for problematic values in training features:")

# Check for infinity
inf_cols = []
for col in X_train_final.columns:
    if np.isinf(X_train_final[col]).any():
        inf_count = np.isinf(X_train_final[col]).sum()
        inf_cols.append((col, inf_count))
        print(f"   ⚠️  {col}: {inf_count} infinity values")

if len(inf_cols) == 0:
    print("   ✓ No infinity values found")

# Check for NaN values
nan_cols = []
for col in X_train_final.columns:
    if X_train_final[col].isna().any():
        nan_count = X_train_final[col].isna().sum()
        nan_cols.append((col, nan_count))
        print(f"   ⚠️  {col}: {nan_count} NaN values")

if len(nan_cols) == 0:
    print("   ✓ No NaN values found")

# Check for extremely large values
print("\n3. Checking for extremely large values:")
large_value_cols = []
for col in X_train_final.columns:
    max_val = X_train_final[col].abs().max()
    if max_val > 1e10:  # Values larger than 10 billion
        large_value_cols.append((col, max_val))
        print(f"   ⚠️  {col}: max absolute value = {max_val:.2e}")

if len(large_value_cols) == 0:
    print("   ✓ No extremely large values found")


# CLEAN THE DATA
print("\n4. CLEANING DATA:")

# Replace infinity with NaN
print("   Replacing infinity values with NaN...")
X_train_final = X_train_final.replace([np.inf, -np.inf], np.nan)
X_val_final = X_val_final.replace([np.inf, -np.inf], np.nan)
X_test_final = X_test_final.replace([np.inf, -np.inf], np.nan)

# Check how many NaN values we have
total_nan = X_train_final.isna().sum().sum()
print(f"   Total NaN values in training set: {total_nan}")

if total_nan > 0:
    # Option 1: Drop columns with too many NaN/inf values (>30%)
    print("\n   Analyzing problematic columns:")
    problematic_cols = []
    for col in X_train_final.columns:
        nan_pct = X_train_final[col].isna().sum() / len(X_train_final) * 100
        if nan_pct > 30:
            problematic_cols.append(col)
            print(f"   - {col}: {nan_pct:.1f}% missing → DROPPING")
    
    if len(problematic_cols) > 0:
        print(f"\n   Dropping {len(problematic_cols)} problematic columns")
        X_train_final = X_train_final.drop(columns=problematic_cols)
        X_val_final = X_val_final.drop(columns=problematic_cols)
        X_test_final = X_test_final.drop(columns=problematic_cols)
    
    # Option 2: Impute remaining NaN values with median
    remaining_nan = X_train_final.isna().sum().sum()
    if remaining_nan > 0:
        print(f"\n   Imputing {remaining_nan} remaining NaN values with column median...")
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy='median')
        
        # Fit on training, transform all
        X_train_final_values = imputer.fit_transform(X_train_final)
        X_val_final_values = imputer.transform(X_val_final)
        X_test_final_values = imputer.transform(X_test_final)
        
        # Convert back to DataFrame
        X_train_final = pd.DataFrame(X_train_final_values, 
                                      columns=X_train_final.columns, 
                                      index=X_train_final.index)
        X_val_final = pd.DataFrame(X_val_final_values, 
                                    columns=X_val_final.columns, 
                                    index=X_val_final.index)
        X_test_final = pd.DataFrame(X_test_final_values, 
                                     columns=X_test_final.columns, 
                                     index=X_test_final.index)
        print("   ✓ Imputation complete")

# Final verification
print("\n5. FINAL VERIFICATION:")
final_inf = np.isinf(X_train_final.values).sum()
final_nan = X_train_final.isna().sum().sum()
print(f"   Training set - Infinity values: {final_inf}")
print(f"   Training set - NaN values: {final_nan}")

if final_inf == 0 and final_nan == 0:
    print("   ✓ Data is clean and ready for scaling!")
else:
    print("   ⚠️  WARNING: Data still has problematic values")

print(f"\nFinal feature count: {X_train_final.shape[1]} features")
print(f"Final observation count: Train={len(X_train_final)}, Val={len(X_val_final)}, Test={len(X_test_final)}")


# 2. FEATURE SCALING
print(f"\n{'='*70}")
print(f"2. FEATURE SCALING (StandardScaler)")
print(f"{'='*70}")

from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit scaler on TRAINING data only (prevent data leakage)
print(f"\nFitting scaler on training data...")
scaler.fit(X_train_final)

# Transform all three sets
X_train_scaled = scaler.transform(X_train_final)
X_val_scaled = scaler.transform(X_val_final)
X_test_scaled = scaler.transform(X_test_final)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_final.columns, index=X_train_final.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val_final.columns, index=X_val_final.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_final.columns, index=X_test_final.index)

print(f"✓ Scaling complete")

# Verify scaling worked (training set should have mean≈0, std≈1)
print(f"\nScaling verification (training set):")
print(f"  Mean of features: {X_train_scaled.mean().mean():.6f} (should be ≈0)")
print(f"  Std of features:  {X_train_scaled.std().mean():.6f} (should be ≈1)")

# Show sample statistics for a few features
print(f"\nSample feature statistics after scaling (training set):")
sample_features = X_train_final.columns[:5]  # First 5 features
for feat in sample_features:
    mean_val = X_train_scaled[feat].mean()
    std_val = X_train_scaled[feat].std()
    print(f"  {feat}: mean={mean_val:.3f}, std={std_val:.3f}")


# 3. SUMMARY
print(f"\n{'='*70}")
print(f"DATA PREPARATION COMPLETE - READY FOR MODELING")
print(f"{'='*70}")

print(f"\nFinal datasets:")
print(f"  Training:   {X_train_scaled.shape[0]} observations × {X_train_scaled.shape[1]} features")
print(f"  Validation: {X_val_scaled.shape[0]} observations × {X_val_scaled.shape[1]} features")
print(f"  Test:       {X_test_scaled.shape[0]} observations × {X_test_scaled.shape[1]} features")

print(f"\nAvailable datasets:")
print(f"  Scaled features:   X_train_scaled, X_val_scaled, X_test_scaled")
print(f"  Unscaled features: X_train_final, X_val_final, X_test_final")
print(f"  Target values:     y_train_final, y_val_final, y_test_final")

print(f"\nNext steps:")
print(f"  - Use SCALED data for: Ridge, Lasso, ElasticNet")
print(f"  - Use UNSCALED data for: OLS regression (with statsmodels)")

print(f"\n{'='*70}")


## Step 5.1: Multicollinearity Diagnostics (VIF Analysis)

After observing the baseline OLS model results, we need to check for **multicollinearity** - when predictor variables are highly correlated with each other. Multicollinearity can cause:

1. **Inflated coefficients** (extremely large positive/negative values)
2. **Unstable estimates** (small data changes cause large coefficient changes)  
3. **Poor generalization** (overfitting to training data)

### Variance Inflation Factor (VIF)

VIF measures how much a feature's variance is inflated due to correlation with other features:

- **VIF = 1**: No correlation with other features
- **VIF = 1-5**: Moderate correlation (acceptable)
- **VIF = 5-10**: High correlation (concerning)
- **VIF > 10**: Severe multicollinearity (problematic)

**Formula**: VIF_i = 1 / (1 - R²_i), where R²_i is from regressing feature i against all others.

In [ ]:
# Step 5.1: Calculate VIF for Baseline Features
print("="*70)
print("STEP 5.1: MULTICOLLINEARITY DIAGNOSTICS (VIF)")
print("="*70)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature (excluding constant)
print("\n1. CALCULATING VIF SCORES:")
print("   VIF measures how much variance is inflated due to multicollinearity")
print("   VIF < 5: Low correlation (good)")
print("   VIF 5-10: Moderate correlation (concerning)")
print("   VIF > 10: Severe multicollinearity (problematic)\n")

# Use the baseline features (without constant)
vif_data = pd.DataFrame()
vif_data["Feature"] = X_train_baseline.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_baseline.values, i) 
                   for i in range(X_train_baseline.shape[1])]

# Sort by VIF (highest first)
vif_data = vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)

# Display VIF scores with color-coded warnings
print(f"{'Feature':<40s} {'VIF':>12s}  {'Status':>15s}")
print("-" * 70)

for idx, row in vif_data.iterrows():
    feat = row['Feature']
    vif = row['VIF']
    
    if vif > 10:
        status = "⚠️  SEVERE"
        color_marker = "🔴"
    elif vif > 5:
        status = "⚠️  MODERATE"
        color_marker = "🟡"
    else:
        status = "✓ GOOD"
        color_marker = "🟢"
    
    print(f"{feat:<40s} {vif:12.2f}  {color_marker} {status}")

# Summary statistics
severe_count = (vif_data['VIF'] > 10).sum()
moderate_count = ((vif_data['VIF'] > 5) & (vif_data['VIF'] <= 10)).sum()
good_count = (vif_data['VIF'] <= 5).sum()

print("\n" + "="*70)
print("2. VIF SUMMARY:")
print("="*70)
print(f"  Features with SEVERE multicollinearity (VIF > 10):   {severe_count}")
print(f"  Features with MODERATE multicollinearity (VIF 5-10): {moderate_count}")
print(f"  Features with LOW multicollinearity (VIF < 5):       {good_count}")
print(f"\n  Average VIF: {vif_data['VIF'].mean():.2f}")
print(f"  Maximum VIF: {vif_data['VIF'].max():.2f}")


# 3. Visualize VIF scores
print("\n" + "="*70)
print("3. VIF VISUALIZATION:")
print("="*70)

fig, ax = plt.subplots(figsize=(12, 8))

# Create color-coded bars
colors = ['#d73027' if vif > 10 else '#fee08b' if vif > 5 else '#1a9850' 
          for vif in vif_data['VIF']]

bars = ax.barh(range(len(vif_data)), vif_data['VIF'], color=colors, edgecolor='black', linewidth=0.5)

# Add threshold lines
ax.axvline(x=5, color='orange', linestyle='--', linewidth=2, label='Moderate threshold (VIF=5)')
ax.axvline(x=10, color='red', linestyle='--', linewidth=2, label='Severe threshold (VIF=10)')

# Formatting
ax.set_yticks(range(len(vif_data)))
ax.set_yticklabels([f.replace('_', ' ')[:35] for f in vif_data['Feature']], fontsize=9)
ax.set_xlabel('Variance Inflation Factor (VIF)', fontsize=11, fontweight='bold')
ax.set_title('Multicollinearity Diagnostics: VIF Scores for Baseline Features', 
             fontsize=13, fontweight='bold')
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3, axis='x')

# Add VIF values as text on bars
for i, (idx, row) in enumerate(vif_data.iterrows()):
    vif = row['VIF']
    ax.text(vif + 0.5, i, f'{vif:.1f}', va='center', fontsize=8)

plt.tight_layout()
plt.show()


print("\n" + "="*70)
print("KEY INSIGHTS:")
print("="*70)

# Identify problematic features
severe_features = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
if severe_features:
    print("\n⚠️  SEVERE MULTICOLLINEARITY DETECTED!")
    print("   Features with VIF > 10:")
    for feat in severe_features:
        vif_val = vif_data[vif_data['Feature'] == feat]['VIF'].values[0]
        print(f"   - {feat}: VIF = {vif_val:.2f}")
    print("\n   These features are highly correlated with other predictors.")
    print("   This explains the inflated coefficients in the OLS model.")
    print("   Regularization (Ridge/Lasso) will help address this issue.")
else:
    print("✓ No severe multicollinearity detected (all VIF < 10)")

print("\n✓ VIF analysis complete! Ready to proceed to regularized models.")

## Step 6: Time Series Modeling with SARIMAX

### Key Insight from Baseline OLS Results

The baseline OLS model revealed **severe overfitting** (Validation R² = -2.19, indicating predictions worse than the mean). This is due to:

1. **Insufficient observations** (42 train obs ÷ 12 features = 3.5 obs/feature, need 10-20)
2. **Severe multicollinearity** (VIF analysis will show highly correlated predictors)
3. **Ignoring temporal structure** - treating time series as cross-sectional data

### Why SARIMAX is the Appropriate Approach

Our data is **quarterly time series** with:
- ✓ **Seasonality**: Quarterly patterns in housing/construction (Q1-Q4)
- ✓ **Trend**: Long-term changes in export volumes
- ✓ **Autocorrelation**: Current values depend on past values
- ✓ **Exogenous predictors**: Economic indicators (housing starts, exchange rates, mortgage rates)

**SARIMAX** = Seasonal AutoRegressive Integrated Moving Average with eXogenous variables

**Model notation**: SARIMAX(p,d,q)(P,D,Q)[s]
- **(p,d,q)**: Non-seasonal components
  - p = AR order (autoregressive - how many past values to use)
  - d = differencing order (to remove trend)
  - q = MA order (moving average - how many past errors to use)
- **(P,D,Q)**: Seasonal components
  - P = seasonal AR order
  - D = seasonal differencing
  - Q = seasonal MA order
- **[s]**: Seasonality period (4 for quarterly data)

### Modeling Strategy

1. **ACF/PACF Analysis**: Identify temporal patterns and estimate initial parameters
2. **Baseline SARIMA**: Build model without exogenous variables (benchmark)
3. **SARIMAX**: Add economic predictors (housing indicators, exchange rates)
4. **Auto ARIMA**: Automatically find optimal (p,d,q)(P,D,Q) parameters
5. **Model Comparison**: Use AIC/BIC and validation performance
6. **Residual Diagnostics**: Check for white noise (Ljung-Box test)
7. **Forecasting**: Evaluate on test set

### Step 6.1: ACF and PACF Analysis

In [ ]:
# Step 6.1: ACF and PACF Analysis for SARIMA Parameter Identification
print("="*70)
print("STEP 6.1: ACF/PACF ANALYSIS")
print("="*70)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

# Prepare time series data (use full dataset before train/val/test split)
# We'll use the imputed data sorted by date
df_ts = df_imputed.sort_values('Date').copy()
df_ts['Date'] = pd.to_datetime(df_ts['Date'])
df_ts = df_ts.set_index('Date')

# Extract target variable
ts_data = df_ts['CAD_Softwood_Export_to_US']

print("\n1. TIME SERIES OVERVIEW:")
print(f"   Time period: {ts_data.index.min()} to {ts_data.index.max()}")
print(f"   Frequency: Quarterly")
print(f"   Number of observations: {len(ts_data)}")
print(f"   Number of years: {len(ts_data)/4:.1f}")


# 2. STATIONARITY TEST (Augmented Dickey-Fuller)
print("\n" + "="*70)
print("2. STATIONARITY TEST (Augmented Dickey-Fuller)")
print("="*70)
print("   H0: Time series has a unit root (non-stationary)")
print("   H1: Time series is stationary\n")

adf_result = adfuller(ts_data.dropna(), autolag='AIC')
print(f"   ADF Statistic: {adf_result[0]:.4f}")
print(f"   p-value: {adf_result[1]:.4f}")
print(f"   Critical values:")
for key, value in adf_result[4].items():
    print(f"      {key}: {value:.4f}")

if adf_result[1] < 0.05:
    print("\n   ✓ STATIONARY: p-value < 0.05, reject H0")
    print("   → Series is stationary (d=0 likely sufficient)")
else:
    print("\n   ⚠️  NON-STATIONARY: p-value >= 0.05, fail to reject H0")
    print("   → Need differencing (d=1 or d=2)")


# 3. ACF AND PACF PLOTS
print("\n" + "="*70)
print("3. ACF AND PACF VISUALIZATION")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Original Time Series
ax1 = axes[0, 0]
ts_data.plot(ax=ax1, linewidth=2, color='#2c7bb6')
ax1.set_title('Original Time Series: Canadian Softwood Exports to US', 
              fontsize=12, fontweight='bold')
ax1.set_xlabel('Date', fontsize=10)
ax1.set_ylabel('Export Volume', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: First Difference (to check if differencing helps)
ax2 = axes[0, 1]
ts_diff = ts_data.diff().dropna()
ts_diff.plot(ax=ax2, linewidth=2, color='#d7191c')
ax2.set_title('First Differenced Series (d=1)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Date', fontsize=10)
ax2.set_ylabel('Change in Exports', fontsize=10)
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax2.grid(True, alpha=0.3)

# Plot 3: ACF (Autocorrelation Function)
ax3 = axes[1, 0]
plot_acf(ts_data.dropna(), lags=20, ax=ax3, alpha=0.05)
ax3.set_title('ACF Plot: Identifies MA(q) order', fontsize=12, fontweight='bold')
ax3.set_xlabel('Lag', fontsize=10)
ax3.set_ylabel('Autocorrelation', fontsize=10)

# Plot 4: PACF (Partial Autocorrelation Function)
ax4 = axes[1, 1]
plot_pacf(ts_data.dropna(), lags=20, ax=ax4, alpha=0.05, method='ywm')
ax4.set_title('PACF Plot: Identifies AR(p) order', fontsize=12, fontweight='bold')
ax4.set_xlabel('Lag', fontsize=10)
ax4.set_ylabel('Partial Autocorrelation', fontsize=10)

plt.tight_layout()
plt.show()


# 4. INTERPRETATION GUIDE
print("\n" + "="*70)
print("4. ACF/PACF INTERPRETATION GUIDE")
print("="*70)

print("\n📊 How to Read ACF/PACF Plots:")
print("   • ACF shows correlation between observation and its lags")
print("   • PACF shows direct correlation, removing intermediate lags")
print("   • Blue shaded area = 95% confidence interval (significance threshold)")
print("   • Spikes outside blue area = statistically significant")

print("\n📐 Parameter Identification:")
print("   NON-SEASONAL COMPONENTS (p,d,q):")
print("   • p (AR order): Number of significant lags in PACF before cutoff")
print("   • d (differencing): 0 if stationary, 1 or 2 if not")
print("   • q (MA order): Number of significant lags in ACF before cutoff")

print("\n   SEASONAL COMPONENTS (P,D,Q)[4]:")
print("   • Look for spikes at seasonal lags (4, 8, 12 for quarterly)")
print("   • P: Significant spike at lag 4 in PACF")
print("   • D: 1 if strong seasonal pattern, 0 otherwise")
print("   • Q: Significant spike at lag 4 in ACF")

print("\n🔍 What to Look For:")
print("   • Gradual decay in ACF → suggests AR component (p > 0)")
print("   • Sharp cutoff in ACF → suggests MA component (q > 0)")
print("   • Sharp cutoff in PACF → confirms AR order")
print("   • Spikes at lags 4,8,12 → seasonal patterns (quarterly)")


# 5. INITIAL PARAMETER SUGGESTIONS
print("\n" + "="*70)
print("5. SUGGESTED STARTING PARAMETERS")
print("="*70)

print("\n   Based on visual inspection of ACF/PACF plots:")
print("   • Examine the plots above to identify:")
print("     - How many lags are significant in PACF? → suggests p")
print("     - How many lags are significant in ACF? → suggests q")
print("     - Is there a spike at lag 4? → suggests P or Q = 1")
print("     - Is series stationary? → determines d")

print("\n   Typical starting values for quarterly data:")
print("   • SARIMA(1,1,1)(1,1,1)[4] - comprehensive seasonal model")
print("   • SARIMA(0,1,1)(0,1,1)[4] - simpler seasonal model")
print("   • SARIMA(1,1,0)(1,1,0)[4] - AR-focused model")

print("\n   We will use auto_arima to systematically test combinations")
print("   and select the model with the lowest AIC/BIC.")

print("\n✓ ACF/PACF analysis complete!")
print("   Ready to build SARIMA models.")

### Step 6.2: Baseline SARIMA Model (No Exogenous Variables)

In [ ]:
# Step 6.2: Baseline SARIMA Model (Auto ARIMA)
print("="*70)
print("STEP 6.2: BASELINE SARIMA MODEL (NO EXOGENOUS VARIABLES)")
print("="*70)

%pip install pmdarima --quiet

from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np

# 1. PREPARE TIME SERIES DATA WITH TRAIN/VAL/TEST SPLIT
print("\n1. DATA PREPARATION:")
print("   Using chronological 60/20/20 split for time series\n")

# Use the imputed data (no feature engineering for SARIMA - it handles lags internally)
ts_full = df_imputed.sort_values('Date').copy()
ts_full['Date'] = pd.to_datetime(ts_full['Date'])
ts_full = ts_full.set_index('Date')

# Extract target variable
y_series = ts_full['CAD_Softwood_Export_to_US'].copy()

# Chronological split (60/20/20)
n_total = len(y_series)
n_train = int(n_total * 0.60)  # 47 observations
n_val = int(n_total * 0.20)    # 16 observations
n_test = n_total - n_train - n_val  # Remaining

# Split data
y_train_ts = y_series.iloc[:n_train]
y_val_ts = y_series.iloc[n_train:n_train+n_val]
y_test_ts = y_series.iloc[n_train+n_val:]

print(f"   Training set:   {y_train_ts.index.min()} to {y_train_ts.index.max()} ({len(y_train_ts)} obs)")
print(f"   Validation set: {y_val_ts.index.min()} to {y_val_ts.index.max()} ({len(y_val_ts)} obs)")
print(f"   Test set:       {y_test_ts.index.min()} to {y_test_ts.index.max()} ({len(y_test_ts)} obs)")


# 2. AUTO ARIMA - AUTOMATICALLY SELECT BEST PARAMETERS
print("\n" + "="*70)
print("2. AUTO ARIMA: FINDING OPTIMAL PARAMETERS")
print("="*70)
print("   Searching for best SARIMA(p,d,q)(P,D,Q)[4] parameters...")
print("   This may take 1-2 minutes...\n")

# Run auto_arima
auto_model = auto_arima(
    y_train_ts,
    start_p=0, max_p=3,           # Test p from 0 to 3
    start_q=0, max_q=3,           # Test q from 0 to 3
    start_P=0, max_P=2,           # Test seasonal P from 0 to 2
    start_Q=0, max_Q=2,           # Test seasonal Q from 0 to 2
    max_d=2, max_D=1,             # Max differencing
    seasonal=True,                # Enable seasonal component
    m=4,                          # Seasonal period = 4 (quarterly)
    trace=True,                   # Print search progress
    error_action='ignore',        # Ignore warnings
    suppress_warnings=True,
    stepwise=True,                # Use stepwise search (faster)
    information_criterion='aic',  # Use AIC for model selection
    n_jobs=-1                     # Use all CPU cores
)

print("\n✓ Auto ARIMA complete!")
print(f"\n   Best Model: SARIMA{auto_model.order}{auto_model.seasonal_order}")
print(f"   AIC: {auto_model.aic():.2f}")
print(f"   BIC: {auto_model.bic():.2f}")


# 3. FIT FINAL MODEL AND MAKE PREDICTIONS
print("\n" + "="*70)
print("3. MODEL EVALUATION")
print("="*70)

# Get in-sample predictions (training set)
train_pred = auto_model.predict_in_sample()

# Forecast validation set
n_periods_val = len(y_val_ts)
val_pred, val_conf_int = auto_model.predict(n_periods=n_periods_val, return_conf_int=True)

# Calculate training metrics
train_mae = mean_absolute_error(y_train_ts, train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train_ts, train_pred))
train_mape = mean_absolute_percentage_error(y_train_ts, train_pred) * 100

print(f"\nTraining Set Performance:")
print(f"  MAE:   {train_mae:,.0f} thousand cubic meters")
print(f"  RMSE:  {train_rmse:,.0f} thousand cubic meters")
print(f"  MAPE:  {train_mape:.2f}%")

# Calculate validation metrics
val_mae = mean_absolute_error(y_val_ts, val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val_ts, val_pred))
val_mape = mean_absolute_percentage_error(y_val_ts, val_pred) * 100

print(f"\nValidation Set Performance:")
print(f"  MAE:   {val_mae:,.0f} thousand cubic meters")
print(f"  RMSE:  {val_rmse:,.0f} thousand cubic meters")
print(f"  MAPE:  {val_mape:.2f}%")

# Generalization check
print(f"\nGeneralization Check:")
mape_diff = val_mape - train_mape
print(f"  MAPE difference (val - train): {mape_diff:.2f}%")
if mape_diff < 5:
    print(f"  ✓ Good generalization (difference < 5%)")
elif mape_diff < 10:
    print(f"  ⚠️  Slight overfitting (difference 5-10%)")
else:
    print(f"  ⚠️  Overfitting detected (difference > 10%)")


# 4. RESIDUAL DIAGNOSTICS
print("\n" + "="*70)
print("4. RESIDUAL DIAGNOSTICS")
print("="*70)

# Get residuals
residuals = auto_model.resid()

# Ljung-Box test for white noise
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)

print("\nLjung-Box Test (Are residuals white noise?):")
print(f"  H0: Residuals are white noise (no autocorrelation)")
print(f"  Test statistic: {lb_test['lb_stat'].values[0]:.4f}")
print(f"  p-value: {lb_test['lb_pvalue'].values[0]:.4f}")

if lb_test['lb_pvalue'].values[0] > 0.05:
    print("  ✓ PASS: Residuals are white noise (p > 0.05)")
    print("  → Model has captured all temporal patterns")
else:
    print("  ⚠️  FAIL: Residuals show autocorrelation (p < 0.05)")
    print("  → Model may be missing some patterns")


# 5. VISUALIZATION
print("\n" + "="*70)
print("5. FORECAST VISUALIZATION")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training + Validation Forecast
ax1 = axes[0, 0]
ax1.plot(y_train_ts.index, y_train_ts.values, label='Training Data', linewidth=2, color='#2c7bb6')
ax1.plot(y_val_ts.index, y_val_ts.values, label='Actual (Validation)', linewidth=2, color='#1a9850')
ax1.plot(y_val_ts.index, val_pred, label='Forecast', linewidth=2, color='#d73027', linestyle='--')
ax1.fill_between(y_val_ts.index, val_conf_int[:, 0], val_conf_int[:, 1], 
                 alpha=0.3, color='#d73027', label='95% Confidence Interval')
ax1.set_title(f'SARIMA{auto_model.order}{auto_model.seasonal_order} Forecast', 
              fontsize=12, fontweight='bold')
ax1.set_xlabel('Date', fontsize=10)
ax1.set_ylabel('Export Volume', fontsize=10)
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Actual vs Predicted (Validation)
ax2 = axes[0, 1]
ax2.scatter(y_val_ts.values, val_pred, alpha=0.7, s=100, edgecolors='black')
min_val = min(y_val_ts.min(), val_pred.min())
max_val = max(y_val_ts.max(), val_pred.max())
ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
ax2.set_title('Actual vs Predicted (Validation Set)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Actual', fontsize=10)
ax2.set_ylabel('Predicted', fontsize=10)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Residuals Over Time
ax3 = axes[1, 0]
ax3.plot(residuals.index, residuals.values, linewidth=1, color='#2c7bb6')
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.set_title('Residuals Over Time', fontsize=12, fontweight='bold')
ax3.set_xlabel('Date', fontsize=10)
ax3.set_ylabel('Residual', fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Residual Histogram
ax4 = axes[1, 1]
ax4.hist(residuals, bins=15, edgecolor='black', alpha=0.7)
ax4.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax4.set_title('Residual Distribution', fontsize=12, fontweight='bold')
ax4.set_xlabel('Residual', fontsize=10)
ax4.set_ylabel('Frequency', fontsize=10)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ Baseline SARIMA model complete!")
print(f"   Best Model: SARIMA{auto_model.order}{auto_model.seasonal_order}")
print(f"   Validation MAPE: {val_mape:.2f}%")

### Step 6.3: SARIMAX with Exogenous Variables

Now we'll add **economic predictors** to improve forecast accuracy. The SARIMAX model uses:
- **Internal patterns**: Autocorrelation (SARIMA component)
- **External drivers**: Economic indicators (exogenous variables)

**Key exogenous variables to include**:
- US housing indicators (housing starts, building permits, new home sales)
- Economic factors (exchange rate, mortgage rates, GDP)
- Canadian housing activity

In [ ]:
# Step 6.3: SARIMAX with Exogenous Variables
print("="*70)
print("STEP 6.3: SARIMAX WITH EXOGENOUS VARIABLES")
print("="*70)

# 1. SELECT EXOGENOUS VARIABLES
print("\n1. SELECTING EXOGENOUS VARIABLES:")
print("   Choosing key economic indicators based on business relevance\n")

# Select meaningful exogenous predictors (no lags - SARIMAX handles that)
exog_features = [
    # US Housing Market (primary drivers of lumber demand)
    'US_Housing_Start',
    'US_Building_Permits',
    'US_New_Home_Sales',
    'US _BP_Single_Housing',
    
    # Economic Factors
    'USCAD_Exchange_Rate',
    'US_Mortgage_Interest_30Y',
    'US_GDP',
    'CAD_GDP',
    
    # Canadian Housing (competing domestic demand)
    'CAD_Housing_Start',
    'CAD_Building Permits'
]

# Filter to features that exist
available_exog = [f for f in exog_features if f in ts_full.columns]
print(f"   Selected {len(available_exog)} exogenous variables:")
for feat in available_exog:
    print(f"   - {feat}")

# Prepare exogenous variable matrices
X_exog = ts_full[available_exog].copy()

# Split exogenous variables (same split as target)
X_train_exog = X_exog.iloc[:n_train].copy()
X_val_exog = X_exog.iloc[n_train:n_train+n_val].copy()
X_test_exog = X_exog.iloc[n_train+n_val:].copy()

# CHECK FOR MISSING VALUES
print(f"\n2. DATA QUALITY CHECK:")
missing_counts = X_train_exog.isnull().sum()
if missing_counts.sum() > 0:
    print("   ⚠️  Missing values detected in exogenous variables:")
    for col in missing_counts[missing_counts > 0].index:
        print(f"   - {col}: {missing_counts[col]} missing")
    print("\n   → Filling missing values with forward fill + backward fill")
    X_train_exog = X_train_exog.fillna(method='ffill').fillna(method='bfill')
    X_val_exog = X_val_exog.fillna(method='ffill').fillna(method='bfill')
    X_test_exog = X_test_exog.fillna(method='ffill').fillna(method='bfill')
else:
    print("   ✓ No missing values in exogenous variables")

# SCALE EXOGENOUS VARIABLES (CRITICAL for SARIMAX!)
print("\n3. SCALING EXOGENOUS VARIABLES:")
print("   Standardizing features (mean=0, std=1) for numerical stability\n")

from sklearn.preprocessing import StandardScaler

scaler_exog = StandardScaler()
X_train_exog_scaled = scaler_exog.fit_transform(X_train_exog)
X_val_exog_scaled = scaler_exog.transform(X_val_exog)
X_test_exog_scaled = scaler_exog.transform(X_test_exog)

# Convert back to DataFrame to preserve column names
X_train_exog_scaled = pd.DataFrame(X_train_exog_scaled, columns=available_exog, index=X_train_exog.index)
X_val_exog_scaled = pd.DataFrame(X_val_exog_scaled, columns=available_exog, index=X_val_exog.index)
X_test_exog_scaled = pd.DataFrame(X_test_exog_scaled, columns=available_exog, index=X_test_exog.index)

print(f"   Scaled data shapes:")
print(f"   Training:   {X_train_exog_scaled.shape}")
print(f"   Validation: {X_val_exog_scaled.shape}")
print(f"   Test:       {X_test_exog_scaled.shape}")

# Verify scaling
print(f"\n   Verification (training set):")
print(f"   Mean (should be ~0): {X_train_exog_scaled.mean().mean():.6f}")
print(f"   Std (should be ~1):  {X_train_exog_scaled.std().mean():.6f}")


# 4. AUTO ARIMA WITH EXOGENOUS VARIABLES
print("\n" + "="*70)
print("4. AUTO ARIMA WITH SCALED EXOGENOUS VARIABLES")
print("="*70)
print("   Searching for best SARIMAX parameters...")
print("   This may take 2-3 minutes...\n")

# Run auto_arima with scaled exogenous variables
try:
    auto_model_X = auto_arima(
        y_train_ts,
        X=X_train_exog_scaled,        # Use SCALED exogenous variables
        start_p=0, max_p=2,           # Reduced range for stability
        start_q=0, max_q=2,
        start_P=0, max_P=1,
        start_Q=0, max_Q=1,
        max_d=2, max_D=1,
        seasonal=True,
        m=4,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
        information_criterion='aic',
        n_jobs=1                      # Use single core for stability
    )
    
    print("\n✓ Auto ARIMA (with exog) complete!")
    print(f"\n   Best Model: SARIMAX{auto_model_X.order}{auto_model_X.seasonal_order}")
    print(f"   AIC: {auto_model_X.aic():.2f}")
    print(f"   BIC: {auto_model_X.bic():.2f}")
    
    sarimax_success = True
    
except Exception as e:
    print(f"\n⚠️  Auto ARIMA with exogenous variables failed: {str(e)[:100]}")
    print("   → This can happen with too many exogenous variables or high multicollinearity")
    print("   → Trying with reduced set of exogenous variables...")
    
    # Try with just top 5 most important features
    reduced_exog = ['US_Housing_Start', 'USCAD_Exchange_Rate', 'US_Mortgage_Interest_30Y', 
                    'US_Building_Permits', 'US_GDP']
    reduced_exog = [f for f in reduced_exog if f in available_exog]
    
    X_train_reduced = X_train_exog_scaled[reduced_exog]
    X_val_reduced = X_val_exog_scaled[reduced_exog]
    
    print(f"\n   Trying with {len(reduced_exog)} key variables:")
    for feat in reduced_exog:
        print(f"   - {feat}")
    
    try:
        auto_model_X = auto_arima(
            y_train_ts,
            X=X_train_reduced,
            start_p=0, max_p=2,
            start_q=0, max_q=2,
            start_P=0, max_P=1,
            start_Q=0, max_Q=1,
            max_d=2, max_D=1,
            seasonal=True,
            m=4,
            trace=True,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True,
            information_criterion='aic',
            n_jobs=1
        )
        
        print("\n✓ Auto ARIMA (with reduced exog) complete!")
        print(f"\n   Best Model: SARIMAX{auto_model_X.order}{auto_model_X.seasonal_order}")
        print(f"   AIC: {auto_model_X.aic():.2f}")
        print(f"   BIC: {auto_model_X.bic():.2f}")
        
        # Update to use reduced set
        X_val_exog_scaled = X_val_reduced
        available_exog = reduced_exog
        sarimax_success = True
        
    except Exception as e2:
        print(f"\n⚠️  Still failed with reduced variables: {str(e2)[:100]}")
        print("   → Will skip SARIMAX and use SARIMA only for comparison")
        sarimax_success = False


# Only proceed if SARIMAX succeeded
if sarimax_success:
    # 5. MODEL EVALUATION
    print("\n" + "="*70)
    print("5. MODEL EVALUATION")
    print("="*70)
    
    # Get in-sample predictions
    train_pred_X = auto_model_X.predict_in_sample(X=X_train_exog_scaled[available_exog] if len(available_exog) < len(X_train_exog_scaled.columns) else X_train_exog_scaled)
    
    # Forecast validation set
    val_pred_X, val_conf_int_X = auto_model_X.predict(
        n_periods=n_periods_val, 
        X=X_val_exog_scaled,
        return_conf_int=True
    )
    
    # Calculate training metrics
    train_mae_X = mean_absolute_error(y_train_ts, train_pred_X)
    train_rmse_X = np.sqrt(mean_squared_error(y_train_ts, train_pred_X))
    train_mape_X = mean_absolute_percentage_error(y_train_ts, train_pred_X) * 100
    
    print(f"\nTraining Set Performance:")
    print(f"  MAE:   {train_mae_X:,.0f} thousand cubic meters")
    print(f"  RMSE:  {train_rmse_X:,.0f} thousand cubic meters")
    print(f"  MAPE:  {train_mape_X:.2f}%")
    
    # Calculate validation metrics
    val_mae_X = mean_absolute_error(y_val_ts, val_pred_X)
    val_rmse_X = np.sqrt(mean_squared_error(y_val_ts, val_pred_X))
    val_mape_X = mean_absolute_percentage_error(y_val_ts, val_pred_X) * 100
    
    print(f"\nValidation Set Performance:")
    print(f"  MAE:   {val_mae_X:,.0f} thousand cubic meters")
    print(f"  RMSE:  {val_rmse_X:,.0f} thousand cubic meters")
    print(f"  MAPE:  {val_mape_X:.2f}%")
    
    # Generalization check
    mape_diff_X = val_mape_X - train_mape_X
    print(f"\nGeneralization Check:")
    print(f"  MAPE difference (val - train): {mape_diff_X:.2f}%")
    if mape_diff_X < 5:
        print(f"  ✓ Good generalization (difference < 5%)")
    elif mape_diff_X < 10:
        print(f"  ⚠️  Slight overfitting (difference 5-10%)")
    else:
        print(f"  ⚠️  Overfitting detected (difference > 10%)")
    
    
    # 6. COMPARE SARIMA VS SARIMAX
    print("\n" + "="*70)
    print("6. MODEL COMPARISON: SARIMA vs SARIMAX")
    print("="*70)
    
    comparison_df = pd.DataFrame({
        'Model': ['SARIMA (no exog)', f'SARIMAX ({len(available_exog)} exog)'],
        'Specification': [
            f"SARIMA{auto_model.order}{auto_model.seasonal_order}",
            f"SARIMAX{auto_model_X.order}{auto_model_X.seasonal_order}"
        ],
        'AIC': [auto_model.aic(), auto_model_X.aic()],
        'BIC': [auto_model.bic(), auto_model_X.bic()],
        'Train_MAPE': [train_mape, train_mape_X],
        'Val_MAPE': [val_mape, val_mape_X],
        'Val_MAE': [val_mae, val_mae_X],
        'Val_RMSE': [val_rmse, val_rmse_X]
    })
    
    print("\n", comparison_df.to_string(index=False))
    
    # Determine best model
    if val_mape_X < val_mape:
        improvement = ((val_mape - val_mape_X) / val_mape) * 100
        print(f"\n✓ SARIMAX WINS!")
        print(f"   Validation MAPE improved by {improvement:.1f}%")
        print(f"   Exogenous variables add predictive value")
        best_model = 'SARIMAX'
        best_model_obj = auto_model_X
        best_val_pred = val_pred_X
        best_val_conf = val_conf_int_X
    else:
        difference = val_mape_X - val_mape
        print(f"\n⚠️  SARIMA WINS")
        print(f"   SARIMAX is {difference:.1f}% worse on validation MAPE")
        print(f"   Exogenous variables may not add value (or cause overfitting)")
        best_model = 'SARIMA'
        best_model_obj = auto_model
        best_val_pred = val_pred
        best_val_conf = val_conf_int
    
    
    # 7. VISUALIZATION
    print("\n" + "="*70)
    print("7. SARIMAX FORECAST VISUALIZATION")
    print("="*70)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: SARIMAX Forecast
    ax1 = axes[0, 0]
    ax1.plot(y_train_ts.index, y_train_ts.values, label='Training Data', linewidth=2, color='#2c7bb6')
    ax1.plot(y_val_ts.index, y_val_ts.values, label='Actual (Validation)', linewidth=2, color='#1a9850')
    ax1.plot(y_val_ts.index, val_pred_X, label='SARIMAX Forecast', linewidth=2, color='#d73027', linestyle='--')
    ax1.fill_between(y_val_ts.index, val_conf_int_X[:, 0], val_conf_int_X[:, 1], 
                     alpha=0.3, color='#d73027', label='95% CI')
    ax1.set_title(f'SARIMAX{auto_model_X.order}{auto_model_X.seasonal_order} with {len(available_exog)} Exog Vars', 
                  fontsize=12, fontweight='bold')
    ax1.set_xlabel('Date', fontsize=10)
    ax1.set_ylabel('Export Volume', fontsize=10)
    ax1.legend(loc='best', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: SARIMA vs SARIMAX Comparison
    ax2 = axes[0, 1]
    ax2.plot(y_val_ts.index, y_val_ts.values, label='Actual', linewidth=2, color='black')
    ax2.plot(y_val_ts.index, val_pred, label=f'SARIMA (MAPE={val_mape:.1f}%)', 
             linewidth=2, color='#2c7bb6', linestyle='--', alpha=0.7)
    ax2.plot(y_val_ts.index, val_pred_X, label=f'SARIMAX (MAPE={val_mape_X:.1f}%)', 
             linewidth=2, color='#d73027', linestyle='--', alpha=0.7)
    ax2.set_title('Model Comparison: SARIMA vs SARIMAX', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Date', fontsize=10)
    ax2.set_ylabel('Export Volume', fontsize=10)
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Actual vs Predicted
    ax3 = axes[1, 0]
    ax3.scatter(y_val_ts.values, val_pred_X, alpha=0.7, s=100, edgecolors='black')
    min_val = min(y_val_ts.min(), val_pred_X.min())
    max_val = max(y_val_ts.max(), val_pred_X.max())
    ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    ax3.set_title('Actual vs Predicted (SARIMAX)', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Actual', fontsize=10)
    ax3.set_ylabel('Predicted', fontsize=10)
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Performance Metrics Bar Chart
    ax4 = axes[1, 1]
    metrics = ['MAPE (%)', 'MAE (÷1000)', 'RMSE (÷1000)']
    sarima_vals = [val_mape, val_mae/1000, val_rmse/1000]
    sarimax_vals = [val_mape_X, val_mae_X/1000, val_rmse_X/1000]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax4.bar(x - width/2, sarima_vals, width, label='SARIMA', color='#2c7bb6')
    ax4.bar(x + width/2, sarimax_vals, width, label='SARIMAX', color='#d73027')
    ax4.set_title('Validation Performance Comparison', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Value', fontsize=10)
    ax4.set_xticks(x)
    ax4.set_xticklabels(metrics, fontsize=9)
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ SARIMAX model complete!")
    print(f"   Best overall model: {best_model}")
    print(f"   Ready for final test set evaluation")

else:
    print("\n⚠️  SARIMAX failed to fit - using SARIMA only")
    best_model = 'SARIMA'
    best_model_obj = auto_model
    best_val_pred = val_pred
    best_val_conf = val_conf_int

### Step 7: Final Model Evaluation on Test Set

Now we evaluate the best model on the **held-out test set** - data the model has never seen. This provides an unbiased estimate of real-world forecast accuracy.

In [ ]:
# Step 7: Final Model Evaluation on Test Set
print("="*70)
print("STEP 7: FINAL MODEL EVALUATION ON TEST SET")
print("="*70)

print(f"\nBest Model Selected: {best_model}")
print(f"   Specification: {best_model_obj.order}{best_model_obj.seasonal_order}")

# 1. FORECAST ON TEST SET
print("\n" + "="*70)
print("1. FORECASTING ON HELD-OUT TEST SET")
print("="*70)
print("   Test set represents future periods the model has never seen")
print(f"   Test period: {y_test_ts.index.min()} to {y_test_ts.index.max()}")
print(f"   Number of periods: {len(y_test_ts)}\n")

# Determine if we need exogenous variables
if best_model == 'SARIMAX':
    print("   Using SARIMAX - providing exogenous variables for test period")
    # Refit model on train + validation combined for final test
    y_train_val_combined = pd.concat([y_train_ts, y_val_ts])
    
    # Get the appropriate exogenous variables
    if 'X_train_reduced' in locals():
        # Use reduced set if that's what worked
        X_train_val_combined = pd.concat([X_train_exog_scaled[available_exog], X_val_exog_scaled])
        X_test_final = X_test_exog_scaled[available_exog]
    else:
        X_train_val_combined = pd.concat([X_train_exog_scaled, X_val_exog_scaled])
        X_test_final = X_test_exog_scaled
    
    # Refit on train+val
    print(f"   Refitting model on combined train+validation ({len(y_train_val_combined)} obs)...")
    final_model = auto_arima(
        y_train_val_combined,
        X=X_train_val_combined,
        start_p=best_model_obj.order[0], max_p=best_model_obj.order[0],  # Use best params
        start_q=best_model_obj.order[2], max_q=best_model_obj.order[2],
        start_P=best_model_obj.seasonal_order[0], max_P=best_model_obj.seasonal_order[0],
        start_Q=best_model_obj.seasonal_order[2], max_Q=best_model_obj.seasonal_order[2],
        d=best_model_obj.order[1],
        D=best_model_obj.seasonal_order[1],
        seasonal=True,
        m=4,
        trace=False,
        error_action='ignore',
        suppress_warnings=True
    )
    
    # Forecast test set
    test_pred, test_conf_int = final_model.predict(
        n_periods=len(y_test_ts),
        X=X_test_final,
        return_conf_int=True
    )
    
else:
    print("   Using SARIMA - no exogenous variables needed")
    # Refit on train + validation combined
    y_train_val_combined = pd.concat([y_train_ts, y_val_ts])
    
    print(f"   Refitting model on combined train+validation ({len(y_train_val_combined)} obs)...")
    final_model = auto_arima(
        y_train_val_combined,
        start_p=best_model_obj.order[0], max_p=best_model_obj.order[0],
        start_q=best_model_obj.order[2], max_q=best_model_obj.order[2],
        start_P=best_model_obj.seasonal_order[0], max_P=best_model_obj.seasonal_order[0],
        start_Q=best_model_obj.seasonal_order[2], max_Q=best_model_obj.seasonal_order[2],
        d=best_model_obj.order[1],
        D=best_model_obj.seasonal_order[1],
        seasonal=True,
        m=4,
        trace=False,
        error_action='ignore',
        suppress_warnings=True
    )
    
    # Forecast test set
    test_pred, test_conf_int = final_model.predict(
        n_periods=len(y_test_ts),
        return_conf_int=True
    )

print("   ✓ Test set forecast complete!")


# 2. CALCULATE TEST SET METRICS
print("\n" + "="*70)
print("2. TEST SET PERFORMANCE METRICS")
print("="*70)

test_mae = mean_absolute_error(y_test_ts, test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test_ts, test_pred))
test_mape = mean_absolute_percentage_error(y_test_ts, test_pred) * 100

print(f"\nFinal Test Set Performance:")
print(f"  MAE:   {test_mae:,.0f} thousand cubic meters")
print(f"  RMSE:  {test_rmse:,.0f} thousand cubic meters")
print(f"  MAPE:  {test_mape:.2f}%")

# Compare with validation performance
print(f"\nComparison to Validation Set:")
if best_model == 'SARIMAX' and sarimax_success:
    print(f"  Validation MAPE: {val_mape_X:.2f}%")
    mape_diff_final = abs(test_mape - val_mape_X)
else:
    print(f"  Validation MAPE: {val_mape:.2f}%")
    mape_diff_final = abs(test_mape - val_mape)

print(f"  Test MAPE:       {test_mape:.2f}%")
print(f"  Difference:      {mape_diff_final:.2f}%")

if mape_diff_final < 3:
    print("  ✓ Consistent performance across validation and test")
elif mape_diff_final < 5:
    print("  ⚠️  Slight difference (but acceptable)")
else:
    print("  ⚠️  Significant difference - model may be sensitive to data splits")


# 3. RESIDUAL DIAGNOSTICS ON TEST SET
print("\n" + "="*70)
print("3. RESIDUAL DIAGNOSTICS")
print("="*70)

# Calculate test residuals
test_residuals = y_test_ts.values - test_pred

# Ljung-Box test for test residuals
lb_test_final = acorr_ljungbox(test_residuals, lags=[5], return_df=True)

print("\nLjung-Box Test (Test Set Residuals):")
print(f"  H0: Residuals are white noise (no autocorrelation)")
print(f"  Test statistic: {lb_test_final['lb_stat'].values[0]:.4f}")
print(f"  p-value: {lb_test_final['lb_pvalue'].values[0]:.4f}")

if lb_test_final['lb_pvalue'].values[0] > 0.05:
    print("  ✓ PASS: Residuals are white noise (p > 0.05)")
    print("  → Model has captured all temporal patterns")
else:
    print("  ⚠️  FAIL: Residuals show autocorrelation (p < 0.05)")
    print("  → Model may be missing some patterns")

# Residual statistics
print("\nResidual Statistics:")
print(f"  Mean:     {test_residuals.mean():,.0f} (should be close to 0)")
print(f"  Std Dev:  {test_residuals.std():,.0f}")
print(f"  Min:      {test_residuals.min():,.0f}")
print(f"  Max:      {test_residuals.max():,.0f}")


# 4. COMPREHENSIVE VISUALIZATION
print("\n" + "="*70)
print("4. COMPREHENSIVE FORECAST VISUALIZATION")
print("="*70)

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Full Time Series with Forecast
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(y_train_ts.index, y_train_ts.values, label='Training Data', linewidth=2, color='#2c7bb6')
ax1.plot(y_val_ts.index, y_val_ts.values, label='Validation Data', linewidth=2, color='#fdae61')
ax1.plot(y_test_ts.index, y_test_ts.values, label='Test Data (Actual)', linewidth=2, color='#1a9850', marker='o')
ax1.plot(y_test_ts.index, test_pred, label='Test Forecast', linewidth=2, color='#d73027', 
         linestyle='--', marker='s')
ax1.fill_between(y_test_ts.index, test_conf_int[:, 0], test_conf_int[:, 1], 
                 alpha=0.2, color='#d73027', label='95% Confidence Interval')
ax1.set_title(f'{best_model} Model: Full Time Series with Test Set Forecast', 
              fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=11)
ax1.set_ylabel('Canadian Softwood Exports (thousand cubic meters)', fontsize=11)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Test Set Forecast Zoom
ax2 = fig.add_subplot(gs[1, 0])
# Show last few validation points for context
context_start = max(0, len(y_val_ts) - 4)
context_dates = y_val_ts.index[context_start:]
context_values = y_val_ts.values[context_start:]
ax2.plot(context_dates, context_values, label='Validation (context)', 
         linewidth=2, color='#fdae61', alpha=0.5)
ax2.plot(y_test_ts.index, y_test_ts.values, label='Actual', linewidth=2.5, 
         color='#1a9850', marker='o', markersize=8)
ax2.plot(y_test_ts.index, test_pred, label='Forecast', linewidth=2.5, 
         color='#d73027', linestyle='--', marker='s', markersize=8)
ax2.fill_between(y_test_ts.index, test_conf_int[:, 0], test_conf_int[:, 1], 
                 alpha=0.3, color='#d73027', label='95% CI')
ax2.set_title(f'Test Set Forecast Detail (MAPE={test_mape:.2f}%)', 
              fontsize=12, fontweight='bold')
ax2.set_xlabel('Date', fontsize=10)
ax2.set_ylabel('Export Volume', fontsize=10)
ax2.legend(loc='best', fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Actual vs Predicted (Test Set)
ax3 = fig.add_subplot(gs[1, 1])
ax3.scatter(y_test_ts.values, test_pred, alpha=0.8, s=150, edgecolors='black', linewidth=1.5)
min_val = min(y_test_ts.min(), test_pred.min())
max_val = max(y_test_ts.max(), test_pred.max())
ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Perfect Prediction')
# Add error bars
for i in range(len(y_test_ts)):
    ax3.plot([y_test_ts.values[i], y_test_ts.values[i]], 
             [y_test_ts.values[i], test_pred[i]], 
             color='gray', alpha=0.3, linewidth=1)
ax3.set_title('Actual vs Predicted (Test Set)', fontsize=12, fontweight='bold')
ax3.set_xlabel('Actual Exports', fontsize=10)
ax3.set_ylabel('Predicted Exports', fontsize=10)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Test Residuals Over Time
ax4 = fig.add_subplot(gs[2, 0])
ax4.plot(y_test_ts.index, test_residuals, linewidth=2, color='#2c7bb6', marker='o')
ax4.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax4.axhline(y=test_residuals.std(), color='orange', linestyle=':', linewidth=1.5, alpha=0.7, label='±1 Std Dev')
ax4.axhline(y=-test_residuals.std(), color='orange', linestyle=':', linewidth=1.5, alpha=0.7)
ax4.set_title('Test Set Residuals Over Time', fontsize=12, fontweight='bold')
ax4.set_xlabel('Date', fontsize=10)
ax4.set_ylabel('Residual (Actual - Predicted)', fontsize=10)
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

# Plot 5: Residual Distribution
ax5 = fig.add_subplot(gs[2, 1])
ax5.hist(test_residuals, bins=8, edgecolor='black', alpha=0.7, color='#2c7bb6')
ax5.axvline(x=0, color='red', linestyle='--', linewidth=2.5, label='Zero')
ax5.axvline(x=test_residuals.mean(), color='orange', linestyle='-', linewidth=2, label=f'Mean={test_residuals.mean():.0f}')
ax5.set_title('Test Set Residual Distribution', fontsize=12, fontweight='bold')
ax5.set_xlabel('Residual', fontsize=10)
ax5.set_ylabel('Frequency', fontsize=10)
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


# 5. FINAL SUMMARY
print("\n" + "="*70)
print("FINAL MODEL SUMMARY")
print("="*70)

print(f"\n📊 Best Model: {best_model}")
print(f"   Specification: {best_model_obj.order}{best_model_obj.seasonal_order}")
if best_model == 'SARIMAX':
    print(f"   Exogenous Variables: {len(available_exog)}")
    print(f"   Variables: {', '.join(available_exog[:5])}{'...' if len(available_exog) > 5 else ''}")

print(f"\n📈 Performance Metrics:")
print(f"   Training MAPE:   {train_mape if best_model == 'SARIMA' else train_mape_X:.2f}%")
print(f"   Validation MAPE: {val_mape if best_model == 'SARIMA' else val_mape_X:.2f}%")
print(f"   Test MAPE:       {test_mape:.2f}%")

print(f"\n✅ Model Quality:")
if test_mape < 10:
    print(f"   Excellent forecast accuracy (MAPE < 10%)")
elif test_mape < 15:
    print(f"   Good forecast accuracy (MAPE < 15%)")
elif test_mape < 20:
    print(f"   Acceptable forecast accuracy (MAPE < 20%)")
else:
    print(f"   Moderate forecast accuracy (MAPE >= 20%)")

print(f"\n🎯 Business Insights:")
print(f"   Average forecast error: ±{test_mae:,.0f} thousand cubic meters")
print(f"   95% of forecasts within: ±{1.96 * test_rmse:,.0f} thousand cubic meters")

print("\n✓ Final model evaluation complete!")
print("   Model is ready for production forecasting")

## Step 8: Detailed Forecast Results Table

View the actual predicted values for each quarter in the test set.

In [ ]:
# Step 8: Detailed Forecast Results
print("="*70)
print("STEP 8: DETAILED FORECAST PREDICTIONS")
print("="*70)

print("\nTarget Variable: Canadian Softwood Lumber Exports to US")
print("Unit: Thousand cubic meters")
print(f"Forecast Period: {y_test_ts.index.min()} to {y_test_ts.index.max()}\n")

# Create detailed forecast table
forecast_results = pd.DataFrame({
    'Date': y_test_ts.index,
    'Quarter': [f"Q{date.quarter} {date.year}" for date in y_test_ts.index],
    'Actual_Exports': y_test_ts.values,
    'Predicted_Exports': test_pred,
    'Lower_95%_CI': test_conf_int[:, 0],
    'Upper_95%_CI': test_conf_int[:, 1],
    'Forecast_Error': y_test_ts.values - test_pred,
    'Absolute_Error': np.abs(y_test_ts.values - test_pred),
    'Percentage_Error': ((y_test_ts.values - test_pred) / y_test_ts.values) * 100
})

print("="*70)
print("FORECAST RESULTS TABLE")
print("="*70)
print("\nAll values in thousand cubic meters\n")

# Display formatted table
print(f"{'Quarter':<12} {'Actual':>15} {'Predicted':>15} {'Error':>15} {'Error %':>10} {'95% CI Range':>30}")
print("-" * 100)

for idx, row in forecast_results.iterrows():
    ci_range = f"[{row['Lower_95%_CI']:,.0f}, {row['Upper_95%_CI']:,.0f}]"
    print(f"{row['Quarter']:<12} {row['Actual_Exports']:>15,.0f} {row['Predicted_Exports']:>15,.0f} "
          f"{row['Forecast_Error']:>15,.0f} {row['Percentage_Error']:>9.2f}% {ci_range:>30}")

# Summary statistics
print("\n" + "="*70)
print("FORECAST SUMMARY STATISTICS")
print("="*70)

print(f"\nAverage Actual Exports:     {forecast_results['Actual_Exports'].mean():>15,.0f} thousand m³")
print(f"Average Predicted Exports:  {forecast_results['Predicted_Exports'].mean():>15,.0f} thousand m³")
print(f"\nMean Absolute Error (MAE):  {forecast_results['Absolute_Error'].mean():>15,.0f} thousand m³")
print(f"Root Mean Squared Error:    {test_rmse:>15,.0f} thousand m³")
print(f"Mean Absolute % Error:      {forecast_results['Percentage_Error'].abs().mean():>14.2f}%")

print(f"\nSmallest Error:  {forecast_results['Absolute_Error'].min():>15,.0f} thousand m³ ({forecast_results.loc[forecast_results['Absolute_Error'].idxmin(), 'Quarter']})")
print(f"Largest Error:   {forecast_results['Absolute_Error'].max():>15,.0f} thousand m³ ({forecast_results.loc[forecast_results['Absolute_Error'].idxmax(), 'Quarter']})")

# Check if actuals fall within confidence intervals
within_ci = ((forecast_results['Actual_Exports'] >= forecast_results['Lower_95%_CI']) & 
             (forecast_results['Actual_Exports'] <= forecast_results['Upper_95%_CI'])).sum()
ci_coverage = (within_ci / len(forecast_results)) * 100

print(f"\nConfidence Interval Coverage: {within_ci}/{len(forecast_results)} ({ci_coverage:.1f}%)")
if ci_coverage >= 90:
    print("  ✓ Good coverage - confidence intervals are reliable")
else:
    print("  ⚠️  Low coverage - confidence intervals may be too narrow")


# Business interpretation
print("\n" + "="*70)
print("BUSINESS INTERPRETATION")
print("="*70)

avg_actual = forecast_results['Actual_Exports'].mean()
avg_predicted = forecast_results['Predicted_Exports'].mean()
bias = avg_predicted - avg_actual

print(f"\n📊 Forecast Accuracy:")
if test_mape < 10:
    print(f"   Excellent - Model achieves {test_mape:.2f}% MAPE")
    print(f"   Forecasts are highly reliable for business planning")
elif test_mape < 15:
    print(f"   Good - Model achieves {test_mape:.2f}% MAPE")
    print(f"   Forecasts are suitable for strategic planning")
else:
    print(f"   Moderate - Model achieves {test_mape:.2f}% MAPE")
    print(f"   Forecasts provide directional guidance")

print(f"\n📈 Forecast Bias:")
if abs(bias) < avg_actual * 0.05:  # Less than 5% bias
    print(f"   Minimal bias ({bias:,.0f} thousand m³)")
    print(f"   Model is well-calibrated (not systematically over/under-predicting)")
elif bias > 0:
    print(f"   Slight over-prediction (+{bias:,.0f} thousand m³ on average)")
    print(f"   Model tends to predict slightly higher than actual")
else:
    print(f"   Slight under-prediction ({bias:,.0f} thousand m³ on average)")
    print(f"   Model tends to predict slightly lower than actual")

print(f"\n🎯 Forecast Range:")
print(f"   Typical forecast: {avg_predicted:,.0f} ± {1.96 * test_rmse:,.0f} thousand m³ (95% CI)")
print(f"   Expected error range: ±{test_mape:.1f}% on average")

print("\n✓ Forecast results complete!")